In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Maryland

## VEST Documentation

Election results from Maryland State Board of Elections (https://elections.maryland.gov/elections/2018/election_data/index.html)  
Precinct shapefile from Maryland Department of Planning

Early, Provisional, and Absentee votes are reported only at the county level. These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within a county.

G18GOVRHOG - Larry Hogan (Republican Party)  
G18GOVDJEA - Ben Jealous (Democratic Party)  
G18GOVLQUI - Shawn Quinn (Libertarian Party)  
G18GOVGSCH - Ian Schlakman (Green Party)  
G18GOVOWRI - Write-in Votes  

G18COMRPHU - Anjali Reed Phukan (Republican Party)  
G18COMDFRA - Peter Franchot (Democratic Party)  
G18COMOWRI - Write-in Votes  

G18ATGRWOL - Craig Wolf (Republican Party)  
G18ATGDFRO - Brian E. Frosh (Democratic Party)  
G18ATGOWRI - Write-in Votes  

G18USSRCAM - Tony Campbell (Republican Party)  
G18USSDCAR - Ben Cardin (Democratic Party)  
G18USSLVOH - Arvin Vohra (Libertarian Party)  
G18USSISIM - Neal Simon (Unaffiliated)  
G18USSOWRI - Write-in Votes  

## Load VEST File

In [2]:
vest_md_18 = gp.read_file("./raw-from-source/VEST/md_2018/md_2018.shp")

In [3]:
data_columns = ['G18COMRPHU', 'G18USSLVOH', 'G18USSDCAR',
       'G18GOVDJEA', 'G18ATGDFRO', 'G18ATGRWOL', 'G18GOVGSCH',
       'G18GOVRHOG', 'G18USSISIM',
       'G18ATGOWRI', 'G18COMOWRI', 'G18GOVOWRI', 'G18USSOWRI', 'G18COMDFRA',
       'G18GOVLQUI', 'G18USSRCAM']

## Load, Clean and Process MD Day Of File

Note: Maryland splits election results between votes on the day of for the precincts and early, provisional, and absentee, which have to be downloaded separately.

In [4]:
#Load file
source_elections = pd.read_csv("./raw-from-source/Election_Results/All_By_Precinct_2018_General.csv")

#Clean the office name
source_elections["Office Name"] = source_elections["Office Name"].str.strip()

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
#Define a function to swap position in a list
def swapPositions(list, pos1, pos2):
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list
 
# Swap the order of two counties (St. Mary's and Somerset - thrown off because of the "St.")
juris_list = swapPositions(sorted(vest_md_18["JURSCODE"].unique()), 18, 19)

In [6]:
#Create a dictionary mapping from county number to the JURSCODE (shortened county name)
num_juris_dict = dict(zip(range(1,25),juris_list))

#Apply the dictionary
source_elections["County"] = source_elections["County"].map(num_juris_dict)

#Filter down to the relevant offices
source_elections = source_elections[source_elections["Office Name"].isin(['Governor / Lt. Governor', 'Comptroller', 'Attorney General','U.S. Senator'])]

#Make a column with the candidate name and the office
source_elections["Cand-Detailed"] = source_elections["Candidate Name"]+"-"+source_elections["Office Name"]

#Make a column that captures the county, election district, and precinct
source_elections["pivot_col"] = source_elections["County"].astype(str)+"-"+source_elections["Election District"].astype(str)+"-"+source_elections["Election Precinct"].astype(str)

#Pivot the data
pivoted_results = pd.pivot_table(source_elections,index=["pivot_col"],columns=["Cand-Detailed"],values=['Election Night Votes'],aggfunc=sum)

#Clean up the columns and indices
pivoted_results = pivoted_results.fillna(0)
pivoted_results.columns = pivoted_results.columns.droplevel(0)
pivoted_results.reset_index(drop=False,inplace=True)

In [7]:
#Define a dictionary to rename the columns
col_rename_dict = {'pivot_col':'County-Dist-Prec',
    'Anjali Reed Phukan-Comptroller':"G18COMRPHU",
       'Arvin Vohra-U.S. Senator':"G18USSLVOH", 'Ben Cardin-U.S. Senator':"G18USSDCAR",
       'Ben Jealous-Governor / Lt. Governor':"G18GOVDJEA",
       'Brian E. Frosh-Attorney General':"G18ATGDFRO", 'Craig Wolf-Attorney General':"G18ATGRWOL",
       'Edward Shlikas-U.S. Senator':"G18USSOWRI_3", 'Ian Schlakman-Governor / Lt. Governor':"G18GOVGSCH",
       'Larry Hogan-Governor / Lt. Governor':"G18GOVRHOG", 'Lih Young-U.S. Senator':"G18USSOWRI_2",
       'Michael B. Puskar-U.S. Senator':"G18USSOWRI_1", 'Neal Simon-U.S. Senator':"G18USSISIM",
       'Other Write-Ins-Attorney General':"G18ATGOWRI",
        'Other Write-Ins-Comptroller':"G18COMOWRI",
       'Other Write-Ins-Governor / Lt. Governor':"G18GOVOWRI",
       'Other Write-Ins-U.S. Senator':"G18USSOWRI", 'Peter Franchot-Comptroller':"G18COMDFRA",
       'Shawn Quinn-Governor / Lt. Governor':"G18GOVLQUI", 'Tony Campbell-U.S. Senator':"G18USSRCAM"}

#Apply the dictionary
pivoted_results.columns = pivoted_results.columns.map(col_rename_dict)

In [8]:
#Clean up a N/A value
pivoted_results.loc[pivoted_results["G18USSOWRI_2"]=='#NULL#','G18USSOWRI_2']=0

#Cast the result columns to ints
for i in pivoted_results.columns:
    if i != "County-Dist-Prec":
        pivoted_results[i] = pivoted_results[i].astype(int)
        
#Combine the USSOWRI columns
for i in range(1,4):
    cand_string = 'G18USSOWRI_'+str(i)
    pivoted_results['G18USSOWRI']+=pivoted_results[cand_string]
    
#Filter down to the relevant columns
pivoted_results = pivoted_results[['County-Dist-Prec', 'G18COMRPHU', 'G18USSLVOH', 'G18USSDCAR',
       'G18GOVDJEA', 'G18ATGDFRO', 'G18ATGRWOL', 'G18GOVGSCH',
       'G18GOVRHOG', 'G18USSISIM',
       'G18ATGOWRI', 'G18COMOWRI', 'G18GOVOWRI', 'G18USSOWRI', 'G18COMDFRA',
       'G18GOVLQUI', 'G18USSRCAM']]

## Load, Clean, and Process the Early, Provisional, and Absentee votes

In [9]:
#Grab the file names
os.listdir("./raw-from-source/Election_Results/Absentee/")

#Loop over the files and create a dataframe out of them
li = []
for file in os.listdir("./raw-from-source/Election_Results/Absentee/"):
    file_path = "./raw-from-source/Election_Results/Absentee/"+file
    county_results = pd.read_csv(file_path)
    #The "len(file)-17",just drops "_2018_General.csv" 
    county_results["County"]=file[0:len(file)-17]
    li.append(county_results)
absentee_results = pd.concat(li, axis=0, ignore_index=True)

#Confirm that all are loaded
absentee_results["County"].value_counts()

Baltimore_County         182
Montgomery_County        162
Prince_Georges_County    152
Anne_Arundel_County      151
Harford_County           117
Frederick_County         109
Howard_County            106
Baltimore_City_County    101
Wicomico_County          101
Carroll_County            94
Charles_County            88
Washington_County         82
Calvert_County            77
St._Marys_County          76
Worcester_County          76
Allegany_County           75
Cecil_County              75
Queen_Annes_County        74
Talbot_County             72
Caroline_County           68
Somerset_County           68
Dorchester_County         68
Kent_County               63
Garrett_County            60
Name: County, dtype: int64

In [10]:
#Clean up the office name column
absentee_results["Office Name"]=absentee_results["Office Name"].str.strip()

#Filter down to relevant races
absentee_results = absentee_results[absentee_results["Office Name"].isin(['Governor / Lt. Governor', 'Comptroller', 'Attorney General','U.S. Senator'])]

#Create a column with candidate name and office name (makes renaming write-ins easier)
absentee_results["Cand-Detailed"] = absentee_results["Candidate Name"]+"-"+absentee_results["Office Name"]

#Use the county as a pivot column
absentee_results["pivot_col"] = absentee_results["County"].astype(str)

#Get the votes that need to be added
absentee_results["Tot_Oth_Votes"]=absentee_results["Total Votes"]-absentee_results["Election Night Votes"]

In [11]:
#Pivot these results
pivoted_abs_results = pd.pivot_table(absentee_results,index=["pivot_col"],columns=["Cand-Detailed"],values=["Tot_Oth_Votes"],aggfunc=sum)

#Clean up the columns and index
pivoted_abs_results = pivoted_abs_results.fillna(0)
pivoted_abs_results.columns = pivoted_abs_results.columns.droplevel(0)
pivoted_abs_results.reset_index(drop=False,inplace=True)

#Rename the columns using the dictionary above
pivoted_abs_results.columns = pivoted_abs_results.columns.map(col_rename_dict)

In [12]:
#Create a dictionary to mape from the county name to the JURS
name_juris_dict = dict(zip(pivoted_abs_results["County-Dist-Prec"].unique(),sorted(vest_md_18["JURSCODE"].unique())))

#Confirm that it's working correctly
display(name_juris_dict)

#Apply the dictionary
pivoted_abs_results["County-Dist-Prec"] = pivoted_abs_results["County-Dist-Prec"].map(name_juris_dict)

{'Allegany_County': 'ALLE',
 'Anne_Arundel_County': 'ANNE',
 'Baltimore_City_County': 'BACI',
 'Baltimore_County': 'BACO',
 'Calvert_County': 'CALV',
 'Caroline_County': 'CARO',
 'Carroll_County': 'CARR',
 'Cecil_County': 'CECI',
 'Charles_County': 'CHAR',
 'Dorchester_County': 'DORC',
 'Frederick_County': 'FRED',
 'Garrett_County': 'GARR',
 'Harford_County': 'HARF',
 'Howard_County': 'HOWA',
 'Kent_County': 'KENT',
 'Montgomery_County': 'MONT',
 'Prince_Georges_County': 'PRIN',
 'Queen_Annes_County': 'QUEE',
 'Somerset_County': 'SOME',
 'St._Marys_County': 'STMA',
 'Talbot_County': 'TALB',
 'Washington_County': 'WASH',
 'Wicomico_County': 'WICO',
 'Worcester_County': 'WORC'}

In [13]:
#Add the other "USSOWRI" votes
for i in range(1,4):
    cand_string = 'G18USSOWRI_'+str(i)
    pivoted_abs_results['G18USSOWRI']+=pivoted_abs_results[cand_string]

#Filter down to the relevant columns
pivoted_abs_results = pivoted_abs_results[['County-Dist-Prec', 'G18COMRPHU', 'G18USSLVOH', 'G18USSDCAR',
       'G18GOVDJEA', 'G18ATGDFRO', 'G18ATGRWOL', 'G18GOVGSCH',
       'G18GOVRHOG', 'G18USSISIM',
       'G18ATGOWRI', 'G18COMOWRI', 'G18GOVOWRI', 'G18USSOWRI', 'G18COMDFRA',
       'G18GOVLQUI', 'G18USSRCAM']]

#Take a look
display(pivoted_abs_results)

Cand-Detailed,County-Dist-Prec,G18COMRPHU,G18USSLVOH,G18USSDCAR,G18GOVDJEA,G18ATGDFRO,G18ATGRWOL,G18GOVGSCH,G18GOVRHOG,G18USSISIM,G18ATGOWRI,G18COMOWRI,G18GOVOWRI,G18USSOWRI,G18COMDFRA,G18GOVLQUI,G18USSRCAM
0,ALLE,1585,38,2332,1296,2082,2448,39,3231,168,6,3,3,6,2836,30,2073
1,ANNE,24760,644,51398,29902,51245,32891,433,53766,3845,48,148,74,71,58150,453,29002
2,BACI,4785,484,56400,42449,55451,6453,486,19152,1756,85,225,57,141,55796,338,3861
3,BACO,26209,800,80277,50465,79746,40901,562,69162,6823,76,178,95,112,91940,491,33614
4,CALV,4196,87,5708,3248,5575,5227,32,7687,368,4,2,8,3,6416,66,4822
5,CARO,1214,31,1422,619,1336,2092,10,2867,183,1,0,3,3,2190,24,1823
6,CARR,7957,190,8195,4063,8243,11573,88,15756,1483,5,15,13,12,11534,119,10142
7,CECI,3975,69,4099,2330,3969,5093,38,6723,345,1,7,3,8,4960,76,4644
8,CHAR,4141,105,14885,10454,14929,5049,74,9335,423,10,25,10,18,15424,66,4708
9,DORC,860,28,2042,1048,1902,1711,16,2616,160,1,3,4,2,2760,20,1461


## Combine the two sets of election results

In [14]:
#Add the two to a list
li = [pivoted_results,pivoted_abs_results]

#Combine the two
joined= pd.concat(li,axis=0,ignore_index=True)

#Take a look
display(joined)

#Split the "County-Dist-Prec" column to get back the different information
joined["County-Dist-Prec"] = joined["County-Dist-Prec"].str.split("-")

#Create a JURSCODE column
joined["JURSCODE"]= np.where((joined["County-Dist-Prec"].str.contains("-")), joined["County-Dist-Prec"].map(lambda x: x[0]), joined["County-Dist-Prec"])

Cand-Detailed,County-Dist-Prec,G18COMRPHU,G18USSLVOH,G18USSDCAR,G18GOVDJEA,G18ATGDFRO,G18ATGRWOL,G18GOVGSCH,G18GOVRHOG,G18USSISIM,G18ATGOWRI,G18COMOWRI,G18GOVOWRI,G18USSOWRI,G18COMDFRA,G18GOVLQUI,G18USSRCAM
0,ALLE-1-0,199,0,53,23,46,260,1,288,10,0,0,0,0,91,4,246
1,ALLE-10-0,314,3,125,38,81,476,0,548,15,0,0,0,0,218,3,422
2,ALLE-11-0,157,5,212,112,192,239,2,322,13,0,0,0,0,270,1,203
3,ALLE-12-0,395,7,402,201,332,598,10,743,34,1,1,2,1,514,6,509
4,ALLE-13-0,364,6,165,61,133,501,3,579,29,0,0,0,0,247,3,446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,STMA,5285,130,5711,3214,5614,6316,52,8732,387,8,10,7,7,6456,94,5867
2054,TALB,2295,53,5144,2232,4895,4442,37,7200,451,1,5,3,3,6976,28,3849
2055,WASH,4687,79,6196,3522,6029,5997,73,8479,409,8,8,9,11,7100,94,5587
2056,WICO,4031,90,6717,4097,6395,5545,49,7870,476,6,6,9,11,7842,88,4856


## Check the state-level totals

In [15]:
statewide_totals_check(vest_md_18,joined,data_columns)

NameError: name 'statewide_totals_check' is not defined

## Check the county-by-county totals

In [ ]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")

In [ ]:
county_totals_check(vest_md_18,joined,data_columns,"JURSCODE",full_print=False)

## Prepare for the precinct vote check

### Define absentee allocation function

In [ ]:
def allocate_absentee(df_receiving_votes,df_allocating,column_list,col_allocating):
    original_cols = list(df_receiving_votes.columns)
    
    #Add in the "Total Votes column"
    df_receiving_votes.loc[:,"Total_Votes"]=0
    for race in column_list:
        df_receiving_votes.loc[:,"Total_Votes"]+=df_receiving_votes.loc[:,race]
    
    #Create the needed dataframes
    precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
    precinct_specific_totals.reset_index(drop=False,inplace=True)
    to_dole_out_totals = pd.DataFrame(df_allocating.groupby([col_allocating]).sum())
    to_dole_out_totals.reset_index(drop=False,inplace=True)
    
    #Print out any instances where the allocation, as written, won't work
    special_allocation_needed = []
    for index, row in precinct_specific_totals.iterrows():
        for race in column_list:
            if (row[race]==0):
                race_district = row[col_allocating]
                if race_district in to_dole_out_totals[col_allocating].unique():
                    to_allocate = int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==race_district][race])
                    if (to_allocate != 0):
                        special_allocation_needed.append([race_district,race])
    
    #Create some new columns for each of these races to deal with the allocation
    for race in column_list:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        df_receiving_votes.loc[:,add_var]=0.0
        df_receiving_votes.loc[:,rem_var]=0.0
        df_receiving_votes.loc[:,floor_var]=0.0

    #Iterate over the rows
    #Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
    for index, row in df_receiving_votes.iterrows():
        if row[col_allocating] in to_dole_out_totals[col_allocating].unique():
            for race in column_list:
                add_var = race+"_add"
                rem_var = race+"_rem"
                floor_var = race+"_floor"
                #Grab the district
                county_id = row[col_allocating]
                if [county_id,race] in special_allocation_needed:
                    #Get the denominator for the allocation - the summed "total votes" for precincts in that grouping
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id]["Total_Votes"]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the "total votes" for this particular precinct
                    val = df_receiving_votes.at[index,"Total_Votes"]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                else:
                    #Get the denominator for the allocation (the precinct vote totals)
                    denom = precinct_specific_totals.loc[precinct_specific_totals[col_allocating]==county_id][race]
                    #Get one of the numerators, how many districtwide votes to allocate
                    numer = to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county_id][race]
                    #Get the vote totals for this race in this precinct
                    val = df_receiving_votes.at[index,race]
                    #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
                if ((float(denom)==0)):
                    vote_share = 0
                else:
                    vote_share = (float(val)/float(denom))*float(numer)
                df_receiving_votes.at[index,add_var] = vote_share
                #Take the decimal remainder of the allocation
                df_receiving_votes.at[index,rem_var] = vote_share%1
                #Take the floor of the allocation
                df_receiving_votes.at[index,floor_var] = np.floor(vote_share)

    #After the first pass through, get the sums of the races by district to assist in the rounding            
    first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())

    #Now we want to iterate district by district to work on rounding
    county_list = list(to_dole_out_totals[col_allocating].unique()) 

    #Iterate over the district
    for county in county_list:
        for race in column_list:
            add_var = race+"_add"
            rem_var = race+"_rem"
            floor_var = race+"_floor"
            #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
            to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals[col_allocating]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
            #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
            for index in df_receiving_votes.loc[df_receiving_votes[col_allocating]==county][rem_var].nlargest(to_go).index:
                df_receiving_votes.at[index,add_var] = np.ceil(df_receiving_votes.at[index,add_var])

    #Iterate over every race again
    for race in column_list:
        add_var = race+"_add"
        #Round every allocation down to not add fractional votes
        df_receiving_votes.loc[:,add_var]=np.floor(df_receiving_votes.loc[:,add_var])
        df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
        
    df_receiving_votes = df_receiving_votes[original_cols]
    
    return df_receiving_votes

In [ ]:
#Separate out the columns that need to be allocated and those receiving votes
joined["to_allocate"] = joined["County-Dist-Prec"].apply(lambda x:len(x))
df_allocating = joined.loc[joined["to_allocate"]==1]
df_receiving_votes = joined.loc[joined["to_allocate"]!=1]

#Allocate the votes
allocated_precinct = allocate_absentee(df_receiving_votes,df_allocating,data_columns,"JURSCODE")

In [ ]:
#Create a join column
allocated_precinct["join_col"]=allocated_precinct["JURSCODE"]+" "+allocated_precinct["County-Dist-Prec"].apply(lambda x:x[1].zfill(2))+"-"+allocated_precinct["County-Dist-Prec"].apply(lambda x:x[2].zfill(3))

In [ ]:
#Deal with duplicate NAMEs in VEST's file
ser = vest_md_18["NAME"].value_counts(dropna=False) 
dup_list = ser[ser>1].index
vest_md_18[vest_md_18["NAME"].isin(dup_list)]

In [ ]:
display(ser[ser>1])

display(vest_md_18[vest_md_18["NAME"].isin(dup_list)])

#Make the appropriate changes (it seems like VEST made a mistake in creating the NAME column here)
vest_md_18.loc[(vest_md_18["NUMBER"]=="21-005") & (vest_md_18["JURSCODE"]=="BACI"),"NAME"]="Baltimore City Precinct 21-005"
vest_md_18.loc[(vest_md_18["NUMBER"]=="08-009") & (vest_md_18["JURSCODE"]=="BACI"),"NAME"]="Baltimore City Precinct 08-009"
vest_md_18.loc[(vest_md_18["NUMBER"]=="20-012") & (vest_md_18["JURSCODE"]=="BACI"),"NAME"]="Baltimore City Precinct 20-012"
vest_md_18.loc[(vest_md_18["NUMBER"]=="08-008") & (vest_md_18["JURSCODE"]=="BACI"),"NAME"]="Baltimore City Precinct 08-008"

In [ ]:
#Create a dictionary of name changes
source_vest_id_changes_dict = {'ALLE 08-000': 'Allegany Precinct 08-000', 'ALLE 29-001': 'Allegany Precinct 29-001', 'ALLE 12-000': 'Allegany Precinct 12-000', 'ALLE 29-002': 'Allegany Precinct 29-002', 'ALLE 07-002': 'Allegany Precinct 07-002', 'ALLE 18-000': 'Allegany Precinct 18-000', 'ALLE 23-000': 'Allegany Precinct 23-000', 'ALLE 07-001': 'Allegany Precinct 07-001', 'ALLE 13-000': 'Allegany Precinct 13-000', 'ALLE 34-000': 'Allegany Precinct 34-000', 'ALLE 20-000': 'Allegany Precinct 20-000', 'ALLE 10-000': 'Allegany Precinct 10-000', 'ALLE 26-000': 'Allegany Precinct 26-000', 'ALLE 16-000': 'Allegany Precinct 16-000', 'ALLE 06-006': 'Allegany Precinct 06-006', 'ALLE 24-000': 'Allegany Precinct 24-000', 'ALLE 04-004': 'Allegany Precinct 04-004', 'ALLE 29-003': 'Allegany Precinct 29-003', 'ALLE 31-000': 'Allegany Precinct 31-000', 'ALLE 22-001': 'Allegany Precinct 22-000', 'ALLE 04-008': 'Allegany Precinct 04-008', 'ALLE 06-001': 'Allegany Precinct 06-001', 'ALLE 09-000': 'Allegany Precinct 09-000', 'ALLE 21-000': 'Allegany Precinct 21-000', 'ALLE 05-004': 'Allegany Precinct 05-004', 'ALLE 11-000': 'Allegany Precinct 11-000', 'ALLE 06-005': 'Allegany Precinct 06-005', 'ALLE 02-000': 'Allegany Precinct 02-000', 'ALLE 03-000': 'Allegany Precinct 03-000', 'ALLE 01-000': 'Allegany Precinct 01-000', 'ALLE 04-003': 'Allegany Precinct 04-003', 'ALLE 04-005': 'Allegany Precinct 04-005', 'ALLE 05-006': 'Allegany Precinct 05-006', 'ALLE 06-003': 'Allegany Precinct 06-003', 'ALLE 04-002': 'Allegany Precinct 04-002', 'ALLE 14-000': 'Allegany Precinct 14-000', 'ALLE 22-002': 'Allegany Precinct 22-001', 'ANNE 03-006': 'ANNE ARUNDEL PRECINCT 03-006', 'ANNE 07-001': 'ANNE ARUNDEL PRECINCT 07-001', 'ANNE 03-011': 'ANNE ARUNDEL PRECINCT 03-011', 'ANNE 02-021': 'ANNE ARUNDEL PRECINCT 02-021', 'ANNE 04-015': 'ANNE ARUNDEL PRECINCT 04-015', 'ANNE 05-026': 'ANNE ARUNDEL PRECINCT 05-026', 'ANNE 05-021': 'ANNE ARUNDEL PRECINCT 05-021', 'ANNE 05-001': 'ANNE ARUNDEL PRECINCT 05-001', 'ANNE 03-012': 'ANNE ARUNDEL PRECINCT 03-012', 'ANNE 03-010': 'ANNE ARUNDEL PRECINCT 03-010', 'ANNE 07-002': 'ANNE ARUNDEL PRECINCT 07-002', 'ANNE 07-006': 'ANNE ARUNDEL PRECINCT 07-006', 'ANNE 03-013': 'ANNE ARUNDEL PRECINCT 03-013', 'ANNE 07-010': 'ANNE ARUNDEL PRECINCT 07-010', 'ANNE 05-017': 'ANNE ARUNDEL PRECINCT 05-017', 'ANNE 03-019': 'ANNE ARUNDEL PRECINCT 03-019', 'ANNE 03-004': 'ANNE ARUNDEL PRECINCT 03-004', 'ANNE 07-021': 'ANNE ARUNDEL PRECINCT 07-021', 'ANNE 03-003': 'ANNE ARUNDEL PRECINCT 03-003', 'ANNE 03-018': 'ANNE ARUNDEL PRECINCT 03-018', 'ANNE 03-005': 'ANNE ARUNDEL PRECINCT 03-005', 'ANNE 06-026': 'ANNE ARUNDEL PRECINCT 06-026', 'ANNE 05-022': 'ANNE ARUNDEL PRECINCT 05-022', 'ANNE 03-017': 'ANNE ARUNDEL PRECINCT 03-017', 'ANNE 01-022': 'ANNE ARUNDEL PRECINCT 01-022', 'ANNE 05-002': 'ANNE ARUNDEL PRECINCT 05-002', 'ANNE 04-004': 'ANNE ARUNDEL PRECINCT 04-004', 'ANNE 05-013': 'ANNE ARUNDEL PRECINCT 05-013', 'ANNE 03-024': 'ANNE ARUNDEL PRECINCT 03-024', 'ANNE 07-007': 'ANNE ARUNDEL PRECINCT 07-007', 'ANNE 07-023': 'ANNE ARUNDEL PRECINCT 07-023', 'ANNE 07-005': 'ANNE ARUNDEL PRECINCT 07-005', 'ANNE 04-023': 'ANNE ARUNDEL PRECINCT 04-023', 'ANNE 07-025': 'ANNE ARUNDEL PRECINCT 07-025', 'ANNE 03-007': 'ANNE ARUNDEL PRECINCT 03-007', 'ANNE 05-004': 'ANNE ARUNDEL PRECINCT 05-004', 'ANNE 02-023': 'ANNE ARUNDEL PRECINCT 02-023', 'ANNE 04-021': 'ANNE ARUNDEL PRECINCT 04-021', 'ANNE 03-001': 'ANNE ARUNDEL PRECINCT 03-001', 'ANNE 03-020': 'ANNE ARUNDEL PRECINCT 03-020', 'ANNE 01-008': 'ANNE ARUNDEL PRECINCT 01-008', 'ANNE 07-013': 'ANNE ARUNDEL PRECINCT 07-013', 'ANNE 03-021': 'ANNE ARUNDEL PRECINCT 03-021', 'ANNE 07-027': 'ANNE ARUNDEL PRECINCT 07-027', 'ANNE 07-022': 'ANNE ARUNDEL PRECINCT 07-022', 'ANNE 05-003': 'ANNE ARUNDEL PRECINCT 05-003', 'ANNE 03-016': 'ANNE ARUNDEL PRECINCT 03-016', 'ANNE 02-022': 'ANNE ARUNDEL PRECINCT 02-022', 'ANNE 06-002': 'ANNE ARUNDEL PRECINCT 06-002', 'ANNE 05-033': 'ANNE ARUNDEL PRECINCT 05-033', 'ANNE 03-023': 'ANNE ARUNDEL PRECINCT 03-023', 'ANNE 05-008': 'ANNE ARUNDEL PRECINCT 05-008', 'ANNE 06-001': 'ANNE ARUNDEL PRECINCT 06-001', 'ANNE 03-015': 'ANNE ARUNDEL PRECINCT 03-015', 'ANNE 07-008': 'ANNE ARUNDEL PRECINCT 07-008', 'ANNE 06-019': 'ANNE ARUNDEL PRECINCT 06-019', 'ANNE 01-001': 'ANNE ARUNDEL PRECINCT 01-001', 'ANNE 05-027': 'ANNE ARUNDEL PRECINCT 05-027', 'ANNE 02-008': 'ANNE ARUNDEL PRECINCT 02-008', 'ANNE 05-011': 'ANNE ARUNDEL PRECINCT 05-011', 'ANNE 02-007': 'ANNE ARUNDEL PRECINCT 02-007', 'ANNE 05-024': 'ANNE ARUNDEL PRECINCT 05-024', 'ANNE 01-002': 'ANNE ARUNDEL PRECINCT 01-002', 'ANNE 04-014': 'ANNE ARUNDEL PRECINCT 04-014', 'ANNE 05-018': 'ANNE ARUNDEL PRECINCT 05-018', 'ANNE 04-008': 'ANNE ARUNDEL PRECINCT 04-008', 'ANNE 03-008': 'ANNE ARUNDEL PRECINCT 03-008', 'ANNE 06-025': 'ANNE ARUNDEL PRECINCT 06-025', 'ANNE 02-010': 'ANNE ARUNDEL PRECINCT 02-010', 'ANNE 03-022': 'ANNE ARUNDEL PRECINCT 03-022', 'ANNE 06-022': 'ANNE ARUNDEL PRECINCT 06-022', 'ANNE 05-020': 'ANNE ARUNDEL PRECINCT 05-020', 'ANNE 05-007': 'ANNE ARUNDEL PRECINCT 05-007', 'ANNE 01-009': 'ANNE ARUNDEL PRECINCT 01-009', 'ANNE 02-020': 'ANNE ARUNDEL PRECINCT 02-020', 'ANNE 07-024': 'ANNE ARUNDEL PRECINCT 07-024', 'ANNE 04-002': 'ANNE ARUNDEL PRECINCT 04-002', 'ANNE 01-007': 'ANNE ARUNDEL PRECINCT 01-007', 'ANNE 04-003': 'ANNE ARUNDEL PRECINCT 04-003', 'ANNE 04-013': 'ANNE ARUNDEL PRECINCT 04-013', 'ANNE 05-005': 'ANNE ARUNDEL PRECINCT 05-005', 'ANNE 07-004': 'ANNE ARUNDEL PRECINCT 07-004', 'ANNE 02-004': 'ANNE ARUNDEL PRECINCT 02-004', 'ANNE 03-014': 'ANNE ARUNDEL PRECINCT 03-014', 'ANNE 01-015': 'ANNE ARUNDEL PRECINCT 01-015', 'ANNE 01-010': 'ANNE ARUNDEL PRECINCT 01-010', 'ANNE 07-003': 'ANNE ARUNDEL PRECINCT 07-003', 'ANNE 06-030': 'ANNE ARUNDEL PRECINCT 06-030', 'ANNE 05-006': 'ANNE ARUNDEL PRECINCT 05-006', 'ANNE 07-009': 'ANNE ARUNDEL PRECINCT 07-009', 'ANNE 01-003': 'ANNE ARUNDEL PRECINCT 01-003', 'ANNE 06-031': 'ANNE ARUNDEL PRECINCT 06-031', 'ANNE 05-019': 'ANNE ARUNDEL PRECINCT 05-019', 'ANNE 05-014': 'ANNE ARUNDEL PRECINCT 05-014', 'ANNE 06-023': 'ANNE ARUNDEL PRECINCT 06-023', 'ANNE 02-003': 'ANNE ARUNDEL PRECINCT 02-003', 'ANNE 06-018': 'ANNE ARUNDEL PRECINCT 06-018', 'ANNE 05-025': 'ANNE ARUNDEL PRECINCT 05-025', 'ANNE 04-009': 'ANNE ARUNDEL PRECINCT 04-009', 'ANNE 01-011': 'ANNE ARUNDEL PRECINCT 01-011', 'ANNE 06-009': 'ANNE ARUNDEL PRECINCT 06-009', 'ANNE 06-032': 'ANNE ARUNDEL PRECINCT 06-032', 'ANNE 02-024': 'ANNE ARUNDEL PRECINCT 02-024', 'ANNE 06-012': 'ANNE ARUNDEL PRECINCT 06-012', 'ANNE 07-026': 'ANNE ARUNDEL PRECINCT 07-026', 'ANNE 03-002': 'ANNE ARUNDEL PRECINCT 03-002', 'ANNE 06-028': 'ANNE ARUNDEL PRECINCT 06-028', 'ANNE 05-012': 'ANNE ARUNDEL PRECINCT 05-012', 'ANNE 07-015': 'ANNE ARUNDEL PRECINCT 07-015', 'ANNE 04-016': 'ANNE ARUNDEL PRECINCT 04-016', 'ANNE 04-020': 'ANNE ARUNDEL PRECINCT 04-020', 'ANNE 07-012': 'ANNE ARUNDEL PRECINCT 07-012', 'ANNE 04-018': 'ANNE ARUNDEL PRECINCT 04-018', 'ANNE 07-018': 'ANNE ARUNDEL PRECINCT 07-018', 'ANNE 06-008': 'ANNE ARUNDEL PRECINCT 06-008', 'ANNE 02-015': 'ANNE ARUNDEL PRECINCT 02-015', 'ANNE 02-001': 'ANNE ARUNDEL PRECINCT 02-001', 'ANNE 01-014': 'ANNE ARUNDEL PRECINCT 01-014', 'ANNE 02-009': 'ANNE ARUNDEL PRECINCT 02-009', 'ANNE 06-024': 'ANNE ARUNDEL PRECINCT 06-024', 'ANNE 02-005': 'ANNE ARUNDEL PRECINCT 02-005', 'ANNE 05-009': 'ANNE ARUNDEL PRECINCT 05-009', 'ANNE 02-013': 'ANNE ARUNDEL PRECINCT 02-013', 'ANNE 01-018': 'ANNE ARUNDEL PRECINCT 01-018', 'ANNE 06-027': 'ANNE ARUNDEL PRECINCT 06-027', 'ANNE 01-019': 'ANNE ARUNDEL PRECINCT 01-019', 'ANNE 02-002': 'ANNE ARUNDEL PRECINCT 02-002', 'ANNE 07-016': 'ANNE ARUNDEL PRECINCT 07-016', 'ANNE 07-019': 'ANNE ARUNDEL PRECINCT 07-019', 'ANNE 07-014': 'ANNE ARUNDEL PRECINCT 07-014', 'ANNE 06-020': 'ANNE ARUNDEL PRECINCT 06-020', 'ANNE 06-029': 'ANNE ARUNDEL PRECINCT 06-029', 'ANNE 01-016': 'ANNE ARUNDEL PRECINCT 01-016', 'ANNE 01-025': 'ANNE ARUNDEL PRECINCT 01-025', 'ANNE 01-020': 'ANNE ARUNDEL PRECINCT 01-020', 'ANNE 05-016': 'ANNE ARUNDEL PRECINCT 05-016', 'ANNE 03-009': 'ANNE ARUNDEL PRECINCT 03-009', 'ANNE 02-018': 'ANNE ARUNDEL PRECINCT 02-018', 'ANNE 01-006': 'ANNE ARUNDEL PRECINCT 01-006', 'ANNE 06-005': 'ANNE ARUNDEL PRECINCT 06-005', 'ANNE 06-017': 'ANNE ARUNDEL PRECINCT 06-017', 'ANNE 05-023': 'ANNE ARUNDEL PRECINCT 05-023', 'ANNE 05-034': 'ANNE ARUNDEL PRECINCT 05-034', 'ANNE 04-011': 'ANNE ARUNDEL PRECINCT 04-011', 'ANNE 02-006': 'ANNE ARUNDEL PRECINCT 02-006', 'ANNE 01-004': 'ANNE ARUNDEL PRECINCT 01-004', 'ANNE 01-005': 'ANNE ARUNDEL PRECINCT 01-005', 'ANNE 05-015': 'ANNE ARUNDEL PRECINCT 05-015', 'ANNE 04-012': 'ANNE ARUNDEL PRECINCT 04-012', 'ANNE 06-021': 'ANNE ARUNDEL PRECINCT 06-021', 'ANNE 02-014': 'ANNE ARUNDEL PRECINCT 02-014', 'ANNE 04-022': 'ANNE ARUNDEL PRECINCT 04-022', 'ANNE 04-007': 'ANNE ARUNDEL PRECINCT 04-007', 'ANNE 02-011': 'ANNE ARUNDEL PRECINCT 02-011', 'ANNE 04-001': 'ANNE ARUNDEL PRECINCT 04-001', 'ANNE 03-025': 'ANNE ARUNDEL PRECINCT 03-025', 'ANNE 07-020': 'ANNE ARUNDEL PRECINCT 07-020', 'ANNE 02-016': 'ANNE ARUNDEL PRECINCT 02-016', 'ANNE 05-030': 'ANNE ARUNDEL PRECINCT 05-030', 'ANNE 02-019': 'ANNE ARUNDEL PRECINCT 02-019', 'ANNE 07-017': 'ANNE ARUNDEL PRECINCT 07-017', 'ANNE 02-012': 'ANNE ARUNDEL PRECINCT 02-012', 'ANNE 01-026': 'ANNE ARUNDEL PRECINCT 01-026', 'ANNE 02-017': 'ANNE ARUNDEL PRECINCT 02-017', 'ANNE 06-010': 'ANNE ARUNDEL PRECINCT 06-010', 'ANNE 04-017': 'ANNE ARUNDEL PRECINCT 04-017', 'ANNE 05-010': 'ANNE ARUNDEL PRECINCT 05-010', 'ANNE 04-005': 'ANNE ARUNDEL PRECINCT 04-005', 'ANNE 05-028': 'ANNE ARUNDEL PRECINCT 05-028', 'ANNE 04-006': 'ANNE ARUNDEL PRECINCT 04-006', 'ANNE 04-024': 'ANNE ARUNDEL PRECINCT 04-024', 'ANNE 01-012': 'ANNE ARUNDEL PRECINCT 01-012', 'ANNE 01-017': 'ANNE ARUNDEL PRECINCT 01-017', 'ANNE 05-029': 'ANNE ARUNDEL PRECINCT 05-029', 'ANNE 01-024': 'ANNE ARUNDEL PRECINCT 01-024', 'ANNE 06-014': 'ANNE ARUNDEL PRECINCT 06-014', 'ANNE 06-011': 'ANNE ARUNDEL PRECINCT 06-011', 'ANNE 07-011': 'ANNE ARUNDEL PRECINCT 07-011', 'ANNE 06-015': 'ANNE ARUNDEL PRECINCT 06-015', 'ANNE 01-021': 'ANNE ARUNDEL PRECINCT 01-021', 'ANNE 04-019': 'ANNE ARUNDEL PRECINCT 04-019', 'ANNE 05-032': 'ANNE ARUNDEL PRECINCT 05-032', 'ANNE 06-013': 'ANNE ARUNDEL PRECINCT 06-013', 'ANNE 06-007': 'ANNE ARUNDEL PRECINCT 06-007', 'ANNE 01-013': 'ANNE ARUNDEL PRECINCT 01-013', 'ANNE 01-023': 'ANNE ARUNDEL PRECINCT 01-023', 'ANNE 06-003': 'ANNE ARUNDEL PRECINCT 06-003', 'ANNE 05-031': 'ANNE ARUNDEL PRECINCT 05-031', 'ANNE 06-016': 'ANNE ARUNDEL PRECINCT 06-016', 'ANNE 04-010': 'ANNE ARUNDEL PRECINCT 04-010', 'ANNE 06-006': 'ANNE ARUNDEL PRECINCT 06-006', 'ANNE 04-026': 'ANNE ARUNDEL PRECINCT 04-026', 'ANNE 06-004': 'ANNE ARUNDEL PRECINCT 06-004', 'ANNE 04-025': 'ANNE ARUNDEL PRECINCT 04-025', 'ANNE 05-035': 'ANNE ARUNDEL PRECINCT 05-035', 'BACI 24-005': 'Baltimore City Precinct 24-005', 'BACI 01-005': 'Baltimore City Precinct 01-005', 'BACI 24-004': 'Baltimore City Precinct 24-004', 'BACI 26-009': 'Baltimore City Precinct 26-009', 'BACI 27-065': 'Baltimore City Precinct 27-065', 'BACI 27-011': 'Baltimore City Precinct 27-011', 'BACI 01-004': 'Baltimore City Precinct 01-004', 'BACI 23-003': 'Baltimore City Precinct 23-003', 'BACI 27-066': 'Baltimore City Precinct 27-066', 'BACI 24-003': 'Baltimore City Precinct 24-003', 'BACI 12-001': 'Baltimore City Precinct 12-001', 'BACI 02-003': 'Baltimore City Precinct 02-003', 'BACI 27-064': 'Baltimore City Precinct 27-064', 'BACI 27-040': 'Baltimore City Precinct 27-040', 'BACI 27-049': 'Baltimore City Precinct 27-049', 'BACI 27-042': 'Baltimore City Precinct 27-042', 'BACI 25-004': 'Baltimore City Precinct 25-004', 'BACI 27-014': 'Baltimore City Precinct 27-014', 'BACI 27-041': 'Baltimore City Precinct 27-041', 'BACI 25-003': 'Baltimore City Precinct 25-003', 'BACI 27-045': 'Baltimore City Precinct 27-045', 'BACI 22-002': 'Baltimore City Precinct 22-002', 'BACI 26-008': 'Baltimore City Precinct 26-008', 'BACI 03-003': 'Baltimore City Precinct 03-003', 'BACI 27-008': 'Baltimore City Precinct 27-008', 'BACI 01-003': 'Baltimore City Precinct 01-003', 'BACI 13-003': 'Baltimore City Precinct 13-003', 'BACI 27-005': 'Baltimore City Precinct 27-005', 'BACI 23-001': 'Baltimore City Precinct 23-001', 'BACI 27-063': 'Baltimore City Precinct 27-063', 'BACI 01-002': 'Baltimore City Precinct 01-002', 'BACI 27-067': 'Baltimore City Precinct 27-067', 'BACI 13-011': 'Baltimore City Precinct 13-011', 'BACI 27-044': 'Baltimore City Precinct 27-044', 'BACI 13-012': 'Baltimore City Precinct 13-012', 'BACI 27-013': 'Baltimore City Precinct 27-013', 'BACI 27-006': 'Baltimore City Precinct 27-006', 'BACI 06-005': 'Baltimore City Precinct 06-005', 'BACI 13-004': 'Baltimore City Precinct 13-004', 'BACI 02-002': 'Baltimore City Precinct 02-002', 'BACI 26-007': 'Baltimore City Precinct 26-007', 'BACI 27-062': 'Baltimore City Precinct 27-062', 'BACI 26-003': 'Baltimore City Precinct 26-003', 'BACI 26-010': 'Baltimore City Precinct 26-010', 'BACI 27-009': 'Baltimore City Precinct 27-009', 'BACI 22-001': 'Baltimore City Precinct 22-001', 'BACI 27-032': 'Baltimore City Precinct 27-032', 'BACI 27-039': 'Baltimore City Precinct 27-039', 'BACI 01-001': 'Baltimore City Precinct 01-001', 'BACI 24-002': 'Baltimore City Precinct 24-002', 'BACI 13-002': 'Baltimore City Precinct 13-002', 'BACI 27-010': 'Baltimore City Precinct 27-010', 'BACI 11-006': 'Baltimore City Precinct 11-006', 'BACI 26-029': 'Baltimore City Precinct 26-029', 'BACI 04-002': 'Baltimore City Precinct 04-002', 'BACI 27-001': 'Baltimore City Precinct 27-001', 'BACI 23-002': 'Baltimore City Precinct 23-002', 'BACI 28-015': 'Baltimore City Precinct 28-015', 'BACI 25-005': 'Baltimore City Precinct 25-005', 'BACI 26-013': 'Baltimore City Precinct 26-013', 'BACI 21-003': 'Baltimore City Precinct 21-003', 'BACI 27-043': 'Baltimore City Precinct 27-043', 'BACI 24-001': 'Baltimore City Precinct 24-001', 'BACI 25-012': 'Baltimore City Precinct 25-012', 'BACI 27-047': 'Baltimore City Precinct 27-047', 'BACI 25-001': 'Baltimore City Precinct 25-001', 'BACI 08-010': 'Baltimore City Precinct 08-010', 'BACI 27-046': 'Baltimore City Precinct 27-046', 'BACI 27-033': 'Baltimore City Precinct 27-033', 'BACI 25-016': 'Baltimore City Precinct 25-016', 'BACI 13-001': 'Baltimore City Precinct 13-001', 'BACI 27-015': 'Baltimore City Precinct 27-015', 'BACI 27-018': 'Baltimore City Precinct 27-018', 'BACI 27-022': 'Baltimore City Precinct 27-022', 'BACI 15-003': 'Baltimore City Precinct 15-003', 'BACI 27-021': 'Baltimore City Precinct 27-021', 'BACI 27-031': 'Baltimore City Precinct 27-031', 'BACI 08-006': 'Baltimore City Precinct 08-006', 'BACI 27-020': 'Baltimore City Precinct 27-020', 'BACI 26-004': 'Baltimore City Precinct 26-004', 'BACI 14-004': 'Baltimore City Precinct 14-004', 'BACI 28-010': 'Baltimore City Precinct 28-010', 'BACI 26-026': 'Baltimore City Precinct 26-026', 'BACI 27-017': 'Baltimore City Precinct 27-017', 'BACI 12-009': 'Baltimore City Precinct 12-009', 'BACI 27-050': 'Baltimore City Precinct 27-050', 'BACI 09-004': 'Baltimore City Precinct 09-004', 'BACI 09-006': 'Baltimore City Precinct 09-006', 'BACI 27-012': 'Baltimore City Precinct 27-012', 'BACI 11-001': 'Baltimore City Precinct 11-001', 'BACI 11-003': 'Baltimore City Precinct 11-003', 'BACI 27-037': 'Baltimore City Precinct 27-037', 'BACI 11-007': 'Baltimore City Precinct 11-007', 'BACI 28-011': 'Baltimore City Precinct 28-011', 'BACI 28-006': 'Baltimore City Precinct 28-006', 'BACI 21-002': 'Baltimore City Precinct 21-002', 'BACI 13-005': 'Baltimore City Precinct 13-005', 'BACI 26-002': 'Baltimore City Precinct 26-002', 'BACI 26-006': 'Baltimore City Precinct 26-006', 'BACI 15-018': 'Baltimore City Precinct 15-018', 'BACI 28-002': 'Baltimore City Precinct 28-002', 'BACI 16-014': 'Baltimore City Precinct 16-014', 'BACI 26-027': 'Baltimore City Precinct 26-027', 'BACI 26-031': 'Baltimore City Precinct 26-031', 'BACI 27-038': 'Baltimore City Precinct 27-038', 'BACI 02-001': 'Baltimore City Precinct 02-001', 'BACI 27-002': 'Baltimore City Precinct 27-002', 'BACI 09-007': 'Baltimore City Precinct 09-007', 'BACI 27-060': 'Baltimore City Precinct 27-060', 'BACI 26-032': 'Baltimore City Precinct 26-032', 'BACI 19-002': 'Baltimore City Precinct 19-002', 'BACI 15-009': 'Baltimore City Precinct 15-009', 'BACI 20-006': 'Baltimore City Precinct 20-006', 'BACI 15-020': 'Baltimore City Precinct 15-020', 'BACI 20-007': 'Baltimore City Precinct 20-007', 'BACI 12-008': 'Baltimore City Precinct 12-008', 'BACI 04-001': 'Baltimore City Precinct 04-001', 'BACI 26-014': 'Baltimore City Precinct 26-014', 'BACI 25-002': 'Baltimore City Precinct 25-002', 'BACI 26-019': 'Baltimore City Precinct 26-019', 'BACI 28-012': 'Baltimore City Precinct 28-012', 'BACI 16-012': 'Baltimore City Precinct 16-012', 'BACI 25-011': 'Baltimore City Precinct 25-011', 'BACI 07-003': 'Baltimore City Precinct 07-003', 'BACI 06-004': 'Baltimore City Precinct 06-004', 'BACI 27-007': 'Baltimore City Precinct 27-007', 'BACI 15-006': 'Baltimore City Precinct 15-006', 'BACI 09-005': 'Baltimore City Precinct 09-005', 'BACI 15-017': 'Baltimore City Precinct 15-017', 'BACI 12-004': 'Baltimore City Precinct 12-004', 'BACI 26-030': 'Baltimore City Precinct 26-030', 'BACI 13-008': 'Baltimore City Precinct 13-008', 'BACI 26-024': 'Baltimore City Precinct 26-024', 'BACI 21-001': 'Baltimore City Precinct 21-001', 'BACI 28-009': 'Baltimore City Precinct 28-009', 'BACI 25-015': 'Baltimore City Precinct 25-015', 'BACI 27-035': 'Baltimore City Precinct 27-035', 'BACI 27-052': 'Baltimore City Precinct 27-052', 'BACI 16-005': 'Baltimore City Precinct 16-005', 'BACI 26-015': 'Baltimore City Precinct 26-015', 'BACI 17-002': 'Baltimore City Precinct 17-002', 'BACI 07-004': 'Baltimore City Precinct 07-004', 'BACI 18-002': 'Baltimore City Precinct 18-002', 'BACI 26-005': 'Baltimore City Precinct 26-005', 'BACI 15-001': 'Baltimore City Precinct 15-001', 'BACI 28-001': 'Baltimore City Precinct 28-001', 'BACI 26-033': 'Baltimore City Precinct 26-033', 'BACI 20-001': 'Baltimore City Precinct 20-001', 'BACI 26-011': 'Baltimore City Precinct 26-011', 'BACI 25-013': 'Baltimore City Precinct 25-013', 'BACI 15-010': 'Baltimore City Precinct 15-010', 'BACI 08-002': 'Baltimore City Precinct 08-002', 'BACI 20-011': 'Baltimore City Precinct 20-011', 'BACI 16-006': 'Baltimore City Precinct 16-006', 'BACI 09-014': 'Baltimore City Precinct 09-014', 'BACI 15-022': 'Baltimore City Precinct 15-022', 'BACI 27-004': 'Baltimore City Precinct 27-004', 'BACI 28-005': 'Baltimore City Precinct 28-005', 'BACI 08-003': 'Baltimore City Precinct 08-003', 'BACI 08-005': 'Baltimore City Precinct 08-005', 'BACI 19-001': 'Baltimore City Precinct 19-001', 'BACI 27-003': 'Baltimore City Precinct 27-003', 'BACI 15-005': 'Baltimore City Precinct 15-005', 'BACI 27-034': 'Baltimore City Precinct 27-034', 'BACI 25-008': 'Baltimore City Precinct 25-008', 'BACI 15-011': 'Baltimore City Precinct 15-011', 'BACI 13-010': 'Baltimore City Precinct 13-010', 'BACI 09-003': 'Baltimore City Precinct 09-003', 'BACI 27-030': 'Baltimore City Precinct 27-030', 'BACI 26-025': 'Baltimore City Precinct 26-025', 'BACI 27-053': 'Baltimore City Precinct 27-053', 'BACI 15-014': 'Baltimore City Precinct 15-014', 'BACI 09-012': 'Baltimore City Precinct 09-012', 'BACI 27-025': 'Baltimore City Precinct 27-025', 'BACI 27-016': 'Baltimore City Precinct 27-016', 'BACI 12-005': 'Baltimore City Precinct 12-005', 'BACI 16-009': 'Baltimore City Precinct 16-009', 'BACI 25-014': 'Baltimore City Precinct 25-014', 'BACI 15-024': 'Baltimore City Precinct 15-024', 'BACI 27-048': 'Baltimore City Precinct 27-048', 'BACI 03-001': 'Baltimore City Precinct 03-001', 'BACI 09-015': 'Baltimore City Precinct 09-015', 'BACI 12-012': 'Baltimore City Precinct 12-012', 'BACI 16-013': 'Baltimore City Precinct 16-013', 'BACI 10-002': 'Baltimore City Precinct 10-002', 'BACI 26-018': 'Baltimore City Precinct 26-018', 'BACI 26-001': 'Baltimore City Precinct 26-001', 'BACI 13-007': 'Baltimore City Precinct 13-007', 'BACI 14-001': 'Baltimore City Precinct 14-001', 'BACI 15-007': 'Baltimore City Precinct 15-007', 'BACI 12-003': 'Baltimore City Precinct 12-003', 'BACI 08-007': 'Baltimore City Precinct 08-007', 'BACI 16-007': 'Baltimore City Precinct 16-007', 'BACI 12-006': 'Baltimore City Precinct 12-006', 'BACI 25-010': 'Baltimore City Precinct 25-010', 'BACI 11-005': 'Baltimore City Precinct 11-005', 'BACI 27-027': 'Baltimore City Precinct 27-027', 'BACI 27-028': 'Baltimore City Precinct 27-028', 'BACI 12-007': 'Baltimore City Precinct 12-007', 'BACI 27-057': 'Baltimore City Precinct 27-057', 'BACI 25-006': 'Baltimore City Precinct 25-006', 'BACI 10-001': 'Baltimore City Precinct 10-001', 'BACI 26-016': 'Baltimore City Precinct 26-016', 'BACI 15-023': 'Baltimore City Precinct 15-023', 'BACI 09-010': 'Baltimore City Precinct 09-010', 'BACI 09-009': 'Baltimore City Precinct 09-009', 'BACI 07-001': 'Baltimore City Precinct 07-001', 'BACI 27-061': 'Baltimore City Precinct 27-061', 'BACI 28-008': 'Baltimore City Precinct 28-008', 'BACI 28-013': 'Baltimore City Precinct 28-013', 'BACI 12-011': 'Baltimore City Precinct 12-011', 'BACI 09-002': 'Baltimore City Precinct 09-002', 'BACI 21-004': 'Baltimore City Precinct 21-004', 'BACI 26-023': 'Baltimore City Precinct 26-023', 'BACI 15-021': 'Baltimore City Precinct 15-021', 'BACI 27-023': 'Baltimore City Precinct 27-023', 'BACI 12-010': 'Baltimore City Precinct 12-010', 'BACI 09-008': 'Baltimore City Precinct 09-008', 'BACI 15-004': 'Baltimore City Precinct 15-004', 'BACI 13-009': 'Baltimore City Precinct 13-009', 'BACI 06-001': 'Baltimore City Precinct 06-001', 'BACI 05-001': 'Baltimore City Precinct 05-001', 'BACI 26-028': 'Baltimore City Precinct 26-028', 'BACI 26-022': 'Baltimore City Precinct 26-022', 'BACI 27-029': 'Baltimore City Precinct 27-029', 'BACI 28-007': 'Baltimore City Precinct 28-007', 'BACI 14-002': 'Baltimore City Precinct 14-002', 'BACI 15-008': 'Baltimore City Precinct 15-008', 'BACI 16-002': 'Baltimore City Precinct 16-002', 'BACI 25-009': 'Baltimore City Precinct 25-009', 'BACI 03-002': 'Baltimore City Precinct 03-002', 'BACI 15-016': 'Baltimore City Precinct 15-016', 'BACI 20-005': 'Baltimore City Precinct 20-005', 'BACI 27-026': 'Baltimore City Precinct 27-026', 'BACI 18-001': 'Baltimore City Precinct 18-001', 'BACI 16-008': 'Baltimore City Precinct 16-008', 'BACI 27-056': 'Baltimore City Precinct 27-056', 'BACI 28-004': 'Baltimore City Precinct 28-004', 'BACI 16-011': 'Baltimore City Precinct 16-011', 'BACI 26-021': 'Baltimore City Precinct 26-021', 'BACI 12-002': 'Baltimore City Precinct 12-002', 'BACI 25-017': 'Baltimore City Precinct 25-017', 'BACI 26-017': 'Baltimore City Precinct 26-017', 'BACI 20-008': 'Baltimore City Precinct 20-008', 'BACI 20-003': 'Baltimore City Precinct 20-003', 'BACI 26-020': 'Baltimore City Precinct 26-020', 'BACI 11-002': 'Baltimore City Precinct 11-002', 'BACI 27-055': 'Baltimore City Precinct 27-055', 'BACI 28-014': 'Baltimore City Precinct 28-014', 'BACI 15-002': 'Baltimore City Precinct 15-002', 'BACI 27-036': 'Baltimore City Precinct 27-036', 'BACI 06-002': 'Baltimore City Precinct 06-002', 'BACI 25-007': 'Baltimore City Precinct 25-007', 'BACI 27-019': 'Baltimore City Precinct 27-019', 'BACI 16-003': 'Baltimore City Precinct 16-003', 'BACI 09-001': 'Baltimore City Precinct 09-001', 'BACI 04-003': 'Baltimore City Precinct 04-003', 'BACI 09-013': 'Baltimore City Precinct 09-013', 'BACI 20-002': 'Baltimore City Precinct 20-002', 'BACI 13-006': 'Baltimore City Precinct 13-006', 'BACI 16-001': 'Baltimore City Precinct 16-001', 'BACI 28-003': 'Baltimore City Precinct 28-003', 'BACI 14-003': 'Baltimore City Precinct 14-003', 'BACI 15-013': 'Baltimore City Precinct 15-013', 'BACI 16-004': 'Baltimore City Precinct 16-004', 'BACI 14-005': 'Baltimore City Precinct 14-005', 'BACI 16-010': 'Baltimore City Precinct 16-010', 'BACI 27-059': 'Baltimore City Precinct 27-059', 'BACI 08-011': 'Baltimore City Precinct 08-011', 'BACI 15-025': 'Baltimore City Precinct 15-025', 'BACI 26-012': 'Baltimore City Precinct 26-012', 'BACI 06-003': 'Baltimore City Precinct 06-003', 'BACI 15-015': 'Baltimore City Precinct 15-015', 'BACI 27-024': 'Baltimore City Precinct 27-024', 'BACI 25-018': 'Baltimore City Precinct 25-018', 'BACI 10-003': 'Baltimore City Precinct 10-003', 'BACI 27-058': 'Baltimore City Precinct 27-058', 'BACI 10-004': 'Baltimore City Precinct 10-004', 'BACI 27-051': 'Baltimore City Precinct 27-051', 'BACI 20-010': 'Baltimore City Precinct 20-010', 'BACI 13-013': 'Baltimore City Precinct 13-013', 'BACI 27-054': 'Baltimore City Precinct 27-054', 'BACI 07-002': 'Baltimore City Precinct 07-002', 'BACI 08-004': 'Baltimore City Precinct 08-004', 'BACI 15-012': 'Baltimore City Precinct 15-012', 'BACI 20-004': 'Baltimore City Precinct 20-004', 'BACI 19-003': 'Baltimore City Precinct 19-003', 'BACI 09-011': 'Baltimore City Precinct 09-011', 'BACI 17-001': 'Baltimore City Precinct 17-001', 'BACI 05-002': 'Baltimore City Precinct 05-002', 'BACI 15-019': 'Baltimore City Precinct 15-019', 'BACI 11-004': 'Baltimore City Precinct 11-004', 'BACI 12-013': 'Baltimore City Precinct 12-013', 'BACI 21-005': 'Baltimore City Precinct 21-005', 'BACI 08-001': 'Baltimore City Precinct 08-001', 'BACI 20-009': 'Baltimore City Precinct 20-009', 'BACI 08-009': 'Baltimore City Precinct 08-009', 'BACI 08-008': 'Baltimore City Precinct 08-008', 'BACI 20-012': 'Baltimore City Precinct 20-012', 'BACO 11-003': 'BALTIMORE PRECINCT 11-003', 'BACO 07-001': 'BALTIMORE PRECINCT 07-001', 'BACO 03-007': 'BALTIMORE PRECINCT 03-007', 'BACO 15-021': 'BALTIMORE PRECINCT 15-021', 'BACO 11-007': 'BALTIMORE PRECINCT 11-007', 'BACO 15-022': 'BALTIMORE PRECINCT 15-022', 'BACO 15-020': 'BALTIMORE PRECINCT 15-020', 'BACO 05-001': 'BALTIMORE PRECINCT 05-001', 'BACO 15-019': 'BALTIMORE PRECINCT 15-019', 'BACO 09-024': 'BALTIMORE PRECINCT 09-024', 'BACO 15-005': 'BALTIMORE PRECINCT 15-005', 'BACO 15-006': 'BALTIMORE PRECINCT 15-006', 'BACO 08-009': 'BALTIMORE PRECINCT 08-009', 'BACO 07-003': 'BALTIMORE PRECINCT 07-003', 'BACO 06-002': 'BALTIMORE PRECINCT 06-002', 'BACO 11-022': 'BALTIMORE PRECINCT 11-022', 'BACO 12-004': 'BALTIMORE PRECINCT 12-004', 'BACO 09-021': 'BALTIMORE PRECINCT 09-021', 'BACO 09-005': 'BALTIMORE PRECINCT 09-005', 'BACO 11-017': 'BALTIMORE PRECINCT 11-017', 'BACO 01-012': 'BALTIMORE PRECINCT 01-012', 'BACO 15-025': 'BALTIMORE PRECINCT 15-025', 'BACO 12-007': 'BALTIMORE PRECINCT 12-007', 'BACO 09-010': 'BALTIMORE PRECINCT 09-010', 'BACO 11-015': 'BALTIMORE PRECINCT 11-015', 'BACO 11-012': 'BALTIMORE PRECINCT 11-012', 'BACO 14-006': 'BALTIMORE PRECINCT 14-006', 'BACO 08-019': 'BALTIMORE PRECINCT 08-019', 'BACO 08-004': 'BALTIMORE PRECINCT 08-004', 'BACO 09-022': 'BALTIMORE PRECINCT 09-022', 'BACO 08-013': 'BALTIMORE PRECINCT 08-013', 'BACO 08-012': 'BALTIMORE PRECINCT 08-012', 'BACO 08-001': 'BALTIMORE PRECINCT 08-001', 'BACO 01-011': 'BALTIMORE PRECINCT 01-011', 'BACO 06-001': 'BALTIMORE PRECINCT 06-001', 'BACO 10-001': 'BALTIMORE PRECINCT 10-001', 'BACO 04-004': 'BALTIMORE PRECINCT 04-004', 'BACO 09-003': 'BALTIMORE PRECINCT 09-003', 'BACO 09-001': 'BALTIMORE PRECINCT 09-001', 'BACO 01-013': 'BALTIMORE PRECINCT 01-013', 'BACO 09-007': 'BALTIMORE PRECINCT 09-007', 'BACO 08-010': 'BALTIMORE PRECINCT 08-010', 'BACO 12-003': 'BALTIMORE PRECINCT 12-003', 'BACO 11-011': 'BALTIMORE PRECINCT 11-011', 'BACO 11-002': 'BALTIMORE PRECINCT 11-002', 'BACO 03-010': 'BALTIMORE PRECINCT 03-010', 'BACO 12-005': 'BALTIMORE PRECINCT 12-005', 'BACO 03-009': 'BALTIMORE PRECINCT 03-009', 'BACO 15-026': 'BALTIMORE PRECINCT 15-026', 'BACO 08-017': 'BALTIMORE PRECINCT 08-017', 'BACO 09-013': 'BALTIMORE PRECINCT 09-013', 'BACO 14-001': 'BALTIMORE PRECINCT 14-001', 'BACO 13-004': 'BALTIMORE PRECINCT 13-004', 'BACO 01-010': 'BALTIMORE PRECINCT 01-010', 'BACO 09-009': 'BALTIMORE PRECINCT 09-009', 'BACO 03-008': 'BALTIMORE PRECINCT 03-008', 'BACO 08-002': 'BALTIMORE PRECINCT 08-002', 'BACO 04-008': 'BALTIMORE PRECINCT 04-008', 'BACO 13-005': 'BALTIMORE PRECINCT 13-005', 'BACO 15-002': 'BALTIMORE PRECINCT 15-002', 'BACO 01-008': 'BALTIMORE PRECINCT 01-008', 'BACO 14-002': 'BALTIMORE PRECINCT 14-002', 'BACO 04-007': 'BALTIMORE PRECINCT 04-007', 'BACO 11-005': 'BALTIMORE PRECINCT 11-005', 'BACO 11-014': 'BALTIMORE PRECINCT 11-014', 'BACO 13-009': 'BALTIMORE PRECINCT 13-009', 'BACO 15-008': 'BALTIMORE PRECINCT 15-008', 'BACO 01-014': 'BALTIMORE PRECINCT 01-014', 'BACO 09-023': 'BALTIMORE PRECINCT 09-023', 'BACO 09-008': 'BALTIMORE PRECINCT 09-008', 'BACO 15-018': 'BALTIMORE PRECINCT 15-018', 'BACO 14-003': 'BALTIMORE PRECINCT 14-003', 'BACO 11-009': 'BALTIMORE PRECINCT 11-009', 'BACO 04-003': 'BALTIMORE PRECINCT 04-003', 'BACO 11-010': 'BALTIMORE PRECINCT 11-010', 'BACO 09-025': 'BALTIMORE PRECINCT 09-025', 'BACO 11-004': 'BALTIMORE PRECINCT 11-004', 'BACO 14-012': 'BALTIMORE PRECINCT 14-012', 'BACO 01-016': 'BALTIMORE PRECINCT 01-016', 'BACO 04-009': 'BALTIMORE PRECINCT 04-009', 'BACO 03-006': 'BALTIMORE PRECINCT 03-006', 'BACO 11-008': 'BALTIMORE PRECINCT 11-008', 'BACO 09-002': 'BALTIMORE PRECINCT 09-002', 'BACO 15-007': 'BALTIMORE PRECINCT 15-007', 'BACO 01-015': 'BALTIMORE PRECINCT 01-015', 'BACO 13-003': 'BALTIMORE PRECINCT 13-003', 'BACO 10-004': 'BALTIMORE PRECINCT 10-004', 'BACO 11-006': 'BALTIMORE PRECINCT 11-006', 'BACO 08-015': 'BALTIMORE PRECINCT 08-015', 'BACO 04-012': 'BALTIMORE PRECINCT 04-012', 'BACO 15-011': 'BALTIMORE PRECINCT 15-011', 'BACO 12-011': 'BALTIMORE PRECINCT 12-011', 'BACO 15-012': 'BALTIMORE PRECINCT 15-012', 'BACO 08-022': 'BALTIMORE PRECINCT 08-022', 'BACO 12-012': 'BALTIMORE PRECINCT 12-012', 'BACO 11-019': 'BALTIMORE PRECINCT 11-019', 'BACO 13-008': 'BALTIMORE PRECINCT 13-008', 'BACO 15-015': 'BALTIMORE PRECINCT 15-015', 'BACO 11-001': 'BALTIMORE PRECINCT 11-001', 'BACO 12-008': 'BALTIMORE PRECINCT 12-008', 'BACO 09-004': 'BALTIMORE PRECINCT 09-004', 'BACO 14-007': 'BALTIMORE PRECINCT 14-007', 'BACO 14-009': 'BALTIMORE PRECINCT 14-009', 'BACO 15-003': 'BALTIMORE PRECINCT 15-003', 'BACO 12-002': 'BALTIMORE PRECINCT 12-002', 'BACO 15-016': 'BALTIMORE PRECINCT 15-016', 'BACO 15-013': 'BALTIMORE PRECINCT 15-013', 'BACO 12-010': 'BALTIMORE PRECINCT 12-010', 'BACO 03-011': 'BALTIMORE PRECINCT 03-011', 'BACO 12-006': 'BALTIMORE PRECINCT 12-006', 'BACO 03-013': 'BALTIMORE PRECINCT 03-013', 'BACO 08-008': 'BALTIMORE PRECINCT 08-008', 'BACO 15-010': 'BALTIMORE PRECINCT 15-010', 'BACO 07-002': 'BALTIMORE PRECINCT 07-002', 'BACO 09-014': 'BALTIMORE PRECINCT 09-014', 'BACO 14-010': 'BALTIMORE PRECINCT 14-010', 'BACO 08-014': 'BALTIMORE PRECINCT 08-014', 'BACO 09-012': 'BALTIMORE PRECINCT 09-012', 'BACO 09-018': 'BALTIMORE PRECINCT 09-018', 'BACO 08-003': 'BALTIMORE PRECINCT 08-003', 'BACO 08-025': 'BALTIMORE PRECINCT 08-025', 'BACO 13-002': 'BALTIMORE PRECINCT 13-002', 'BACO 09-019': 'BALTIMORE PRECINCT 09-019', 'BACO 14-008': 'BALTIMORE PRECINCT 14-008', 'BACO 12-001': 'BALTIMORE PRECINCT 12-001', 'BACO 01-004': 'BALTIMORE PRECINCT 01-004', 'BACO 09-015': 'BALTIMORE PRECINCT 09-015', 'BACO 10-002': 'BALTIMORE PRECINCT 10-002', 'BACO 08-007': 'BALTIMORE PRECINCT 08-007', 'BACO 01-009': 'BALTIMORE PRECINCT 01-009', 'BACO 08-006': 'BALTIMORE PRECINCT 08-006', 'BACO 01-003': 'BALTIMORE PRECINCT 01-003', 'BACO 04-002': 'BALTIMORE PRECINCT 04-002', 'BACO 15-017': 'BALTIMORE PRECINCT 15-017', 'BACO 09-011': 'BALTIMORE PRECINCT 09-011', 'BACO 03-005': 'BALTIMORE PRECINCT 03-005', 'BACO 14-011': 'BALTIMORE PRECINCT 14-011', 'BACO 15-004': 'BALTIMORE PRECINCT 15-004', 'BACO 04-005': 'BALTIMORE PRECINCT 04-005', 'BACO 08-020': 'BALTIMORE PRECINCT 08-020', 'BACO 10-003': 'BALTIMORE PRECINCT 10-003', 'BACO 15-009': 'BALTIMORE PRECINCT 15-009', 'BACO 04-006': 'BALTIMORE PRECINCT 04-006', 'BACO 13-001': 'BALTIMORE PRECINCT 13-001', 'BACO 01-005': 'BALTIMORE PRECINCT 01-005', 'BACO 08-027': 'BALTIMORE PRECINCT 08-027', 'BACO 08-024': 'BALTIMORE PRECINCT 08-024', 'BACO 14-013': 'BALTIMORE PRECINCT 14-013', 'BACO 09-026': 'BALTIMORE PRECINCT 09-026', 'BACO 08-018': 'BALTIMORE PRECINCT 08-018', 'BACO 08-011': 'BALTIMORE PRECINCT 08-011', 'BACO 02-019': 'BALTIMORE PRECINCT 02-019', 'BACO 02-021': 'BALTIMORE PRECINCT 02-021', 'BACO 04-001': 'BALTIMORE PRECINCT 04-001', 'BACO 02-015': 'BALTIMORE PRECINCT 02-015', 'BACO 09-016': 'BALTIMORE PRECINCT 09-016', 'BACO 11-023': 'BALTIMORE PRECINCT 11-023', 'BACO 03-014': 'BALTIMORE PRECINCT 03-014', 'BACO 13-007': 'BALTIMORE PRECINCT 13-007', 'BACO 01-006': 'BALTIMORE PRECINCT 01-006', 'BACO 09-017': 'BALTIMORE PRECINCT 09-017', 'BACO 01-007': 'BALTIMORE PRECINCT 01-007', 'BACO 04-010': 'BALTIMORE PRECINCT 04-010', 'BACO 03-004': 'BALTIMORE PRECINCT 03-004', 'BACO 11-016': 'BALTIMORE PRECINCT 11-016', 'BACO 15-001': 'BALTIMORE PRECINCT 15-001', 'BACO 01-017': 'BALTIMORE PRECINCT 01-017', 'BACO 03-001': 'BALTIMORE PRECINCT 03-001', 'BACO 03-012': 'BALTIMORE PRECINCT 03-012', 'BACO 08-016': 'BALTIMORE PRECINCT 08-016', 'BACO 15-014': 'BALTIMORE PRECINCT 15-014', 'BACO 03-002': 'BALTIMORE PRECINCT 03-002', 'BACO 15-023': 'BALTIMORE PRECINCT 15-023', 'BACO 02-012': 'BALTIMORE PRECINCT 02-012', 'BACO 02-011': 'BALTIMORE PRECINCT 02-011', 'BACO 02-026': 'BALTIMORE PRECINCT 02-026', 'BACO 08-023': 'BALTIMORE PRECINCT 08-023', 'BACO 01-001': 'BALTIMORE PRECINCT 01-001', 'BACO 02-013': 'BALTIMORE PRECINCT 02-013', 'BACO 02-010': 'BALTIMORE PRECINCT 02-010', 'BACO 05-002': 'BALTIMORE PRECINCT 05-002', 'BACO 02-014': 'BALTIMORE PRECINCT 02-014', 'BACO 02-029': 'BALTIMORE PRECINCT 02-029', 'BACO 03-003': 'BALTIMORE PRECINCT 03-003', 'BACO 02-017': 'BALTIMORE PRECINCT 02-017', 'BACO 13-006': 'BALTIMORE PRECINCT 13-006', 'BACO 09-028': 'BALTIMORE PRECINCT 09-028', 'BACO 09-006': 'BALTIMORE PRECINCT 09-006', 'BACO 09-020': 'BALTIMORE PRECINCT 09-020', 'BACO 04-013': 'BALTIMORE PRECINCT 04-013', 'BACO 02-008': 'BALTIMORE PRECINCT 02-008', 'BACO 02-006': 'BALTIMORE PRECINCT 02-006', 'BACO 08-005': 'BALTIMORE PRECINCT 08-005', 'BACO 02-025': 'BALTIMORE PRECINCT 02-025', 'BACO 04-014': 'BALTIMORE PRECINCT 04-014', 'BACO 08-021': 'BALTIMORE PRECINCT 08-021', 'BACO 11-018': 'BALTIMORE PRECINCT 11-018', 'BACO 02-027': 'BALTIMORE PRECINCT 02-027', 'BACO 15-024': 'BALTIMORE PRECINCT 15-024', 'BACO 11-013': 'BALTIMORE PRECINCT 11-013', 'BACO 02-024': 'BALTIMORE PRECINCT 02-024', 'BACO 02-004': 'BALTIMORE PRECINCT 02-004', 'BACO 10-007': 'BALTIMORE PRECINCT 10-007', 'BACO 02-031': 'BALTIMORE PRECINCT 02-031', 'BACO 12-009': 'BALTIMORE PRECINCT 12-009', 'BACO 02-005': 'BALTIMORE PRECINCT 02-005', 'BACO 02-018': 'BALTIMORE PRECINCT 02-018', 'BACO 02-007': 'BALTIMORE PRECINCT 02-007', 'BACO 04-011': 'BALTIMORE PRECINCT 04-011', 'BACO 11-021': 'BALTIMORE PRECINCT 11-021', 'BACO 08-028': 'BALTIMORE PRECINCT 08-028', 'BACO 11-026': 'BALTIMORE PRECINCT 11-026', 'BACO 10-005': 'BALTIMORE PRECINCT 10-005', 'BACO 02-020': 'BALTIMORE PRECINCT 02-020', 'BACO 01-002': 'BALTIMORE PRECINCT 01-002', 'BACO 02-002': 'BALTIMORE PRECINCT 02-002', 'BACO 02-009': 'BALTIMORE PRECINCT 02-009', 'BACO 02-001': 'BALTIMORE PRECINCT 02-001', 'BACO 14-014': 'BALTIMORE PRECINCT 14-014', 'BACO 02-016': 'BALTIMORE PRECINCT 02-016', 'BACO 14-004': 'BALTIMORE PRECINCT 14-004', 'BACO 11-024': 'BALTIMORE PRECINCT 11-024', 'BACO 14-005': 'BALTIMORE PRECINCT 14-005', 'BACO 02-030': 'BALTIMORE PRECINCT 02-030', 'BACO 02-023': 'BALTIMORE PRECINCT 02-023', 'BACO 12-013': 'BALTIMORE PRECINCT 12-013', 'BACO 04-015': 'BALTIMORE PRECINCT 04-015', 'BACO 02-003': 'BALTIMORE PRECINCT 02-003', 'BACO 11-020': 'BALTIMORE PRECINCT 11-020', 'BACO 02-028': 'BALTIMORE PRECINCT 02-028', 'BACO 02-022': 'BALTIMORE PRECINCT 02-022', 'BACO 09-029': 'BALTIMORE PRECINCT 09-029', 'BACO 08-026': 'BALTIMORE PRECINCT 08-026', 'BACO 10-006': 'BALTIMORE PRECINCT 09-027', 'BACO 09-027': 'BALTIMORE PRECINCT 10-006', 'BACO 11-027': 'BALTIMORE PRECINCT 11-027', 'CALV 03-008': 'CALVERT PRECINCT 03-008', 'CALV 03-007': 'CALVERT PRECINCT 03-007', 'CALV 03-006': 'CALVERT PRECINCT 03-006', 'CALV 02-005': 'CALVERT PRECINCT 02-005', 'CALV 01-007': 'CALVERT PRECINCT 01-007', 'CALV 02-001': 'CALVERT PRECINCT 02-001', 'CALV 03-004': 'CALVERT PRECINCT 03-004', 'CALV 01-001': 'CALVERT PRECINCT 01-001', 'CALV 02-008': 'CALVERT PRECINCT 02-008', 'CALV 03-005': 'CALVERT PRECINCT 03-005', 'CALV 03-003': 'CALVERT PRECINCT 03-003', 'CALV 02-007': 'CALVERT PRECINCT 02-007', 'CALV 01-005': 'CALVERT PRECINCT 01-005', 'CALV 02-003': 'CALVERT PRECINCT 02-003', 'CALV 01-004': 'CALVERT PRECINCT 01-004', 'CALV 02-002': 'CALVERT PRECINCT 02-002', 'CALV 01-006': 'CALVERT PRECINCT 01-006', 'CALV 02-006': 'CALVERT PRECINCT 02-006', 'CALV 01-003': 'CALVERT PRECINCT 01-003', 'CALV 03-002': 'CALVERT PRECINCT 03-002', 'CALV 02-004': 'CALVERT PRECINCT 02-004', 'CALV 03-001': 'CALVERT PRECINCT 03-001', 'CALV 01-002': 'CALVERT PRECINCT 01-002', 'CARO 02-001': 'Caroline District 02-001', 'CARO 03-001': 'Caroline District 03-001', 'CARO 04-001': 'Caroline District 04-001', 'CARO 07-001': 'Caroline District 07-001', 'CARO 05-001': 'Caroline District 05-001', 'CARO 08-001': 'Caroline District 08-001', 'CARO 01-001': 'Caroline District 01-001', 'CARO 06-001': 'Caroline District 06-001', 'CARR 13-001': 'CARROLL PRECINCT 13-001', 'CARR 05-004': 'CARROLL PRECINCT 05-004', 'CARR 04-001': 'CARROLL PRECINCT 04-001', 'CARR 06-001': 'CARROLL PRECINCT 06-001', 'CARR 14-002': 'CARROLL PRECINCT 14-002', 'CARR 05-002': 'CARROLL PRECINCT 05-002', 'CARR 06-002': 'CARROLL PRECINCT 06-002', 'CARR 08-001': 'CARROLL PRECINCT 08-001', 'CARR 07-004': 'CARROLL PRECINCT 07-004', 'CARR 04-003': 'CARROLL PRECINCT 04-003', 'CARR 07-007': 'CARROLL PRECINCT 07-007', 'CARR 03-001': 'CARROLL PRECINCT 03-001', 'CARR 01-001': 'CARROLL PRECINCT 01-001', 'CARR 09-001': 'CARROLL PRECINCT 09-001', 'CARR 08-002': 'CARROLL PRECINCT 08-002', 'CARR 05-001': 'CARROLL PRECINCT 05-001', 'CARR 14-001': 'CARROLL PRECINCT 14-001', 'CARR 05-003': 'CARROLL PRECINCT 05-003', 'CARR 08-003': 'CARROLL PRECINCT 08-003', 'CARR 07-002': 'CARROLL PRECINCT 07-002', 'CARR 11-001': 'CARROLL PRECINCT 11-001', 'CARR 05-005': 'CARROLL PRECINCT 05-005', 'CARR 01-002': 'CARROLL PRECINCT 01-002', 'CARR 04-002': 'CARROLL PRECINCT 04-002', 'CARR 07-003': 'CARROLL PRECINCT 07-003', 'CARR 07-006': 'CARROLL PRECINCT 07-006', 'CARR 02-002': 'CARROLL PRECINCT 02-002', 'CARR 07-008': 'CARROLL PRECINCT 07-008', 'CARR 09-002': 'CARROLL PRECINCT 09-002', 'CARR 02-001': 'CARROLL PRECINCT 02-001', 'CARR 07-001': 'CARROLL PRECINCT 07-001', 'CARR 10-001': 'CARROLL PRECINCT 10-001', 'CARR 07-005': 'CARROLL PRECINCT 07-005', 'CARR 12-001': 'CARROLL PRECINCT 12-001', 'CARR 14-003': 'CARROLL PRECINCT 14-003', 'CARR 05-006': 'CARROLL PRECINCT 05-006', 'CECI 05-003': 'CECIL PRECINCT 05-003', 'CECI 06-001': 'CECIL PRECINCT 06-001', 'CECI 06-002': 'CECIL PRECINCT 06-002', 'CECI 04-001': 'CECIL PRECINCT 04-001', 'CECI 01-001': 'CECIL PRECINCT 01-001', 'CECI 05-005': 'CECIL PRECINCT 05-005', 'CECI 07-001': 'CECIL PRECINCT 07-001', 'CECI 07-002': 'CECIL PRECINCT 07-002', 'CECI 08-001': 'CECIL PRECINCT 08-001', 'CECI 09-001': 'CECIL PRECINCT 09-001', 'CECI 05-002': 'CECIL PRECINCT 05-002', 'CECI 03-001': 'CECIL PRECINCT 03-001', 'CECI 03-006': 'CECIL PRECINCT 03-006', 'CECI 03-003': 'CECIL PRECINCT 03-003', 'CECI 02-001': 'CECIL PRECINCT 02-001', 'CECI 05-001': 'CECIL PRECINCT 05-001', 'CECI 02-002': 'CECIL PRECINCT 02-002', 'CECI 09-002': 'CECIL PRECINCT 09-002', 'CECI 05-004': 'CECIL PRECINCT 05-004', 'CECI 03-002': 'CECIL PRECINCT 03-002', 'CECI 03-007': 'CECIL PRECINCT 03-007', 'CECI 04-002': 'CECIL PRECINCT 04-002', 'CECI 06-003': 'CECIL PRECINCT 06-003', 'CECI 07-003': 'CECIL PRECINCT 07-003', 'CECI 05-006': 'CECIL PRECINCT 05-006', 'CECI 03-005': 'CECIL PRECINCT 03-005', 'CECI 07-004': 'CECIL PRECINCT 07-004', 'CECI 03-004': 'CECIL PRECINCT 03-004', 'CHAR 08-001': 'CHARLES PRECINCT 08-001', 'CHAR 05-001': 'CHARLES PRECINCT 05-001', 'CHAR 04-001': 'CHARLES PRECINCT 04-001', 'CHAR 01-002': 'CHARLES PRECINCT 01-002', 'CHAR 09-002': 'CHARLES PRECINCT 09-002', 'CHAR 01-003': 'CHARLES PRECINCT 01-003', 'CHAR 08-004': 'CHARLES PRECINCT 08-004', 'CHAR 06-002': 'CHARLES PRECINCT 06-002', 'CHAR 10-001': 'CHARLES PRECINCT 10-001', 'CHAR 01-004': 'CHARLES PRECINCT 01-004', 'CHAR 06-001': 'CHARLES PRECINCT 06-001', 'CHAR 06-014': 'CHARLES PRECINCT 06-014', 'CHAR 09-001': 'CHARLES PRECINCT 09-001', 'CHAR 03-001': 'CHARLES PRECINCT 03-001', 'CHAR 04-002': 'CHARLES PRECINCT 04-002', 'CHAR 01-001': 'CHARLES PRECINCT 01-001', 'CHAR 06-006': 'CHARLES PRECINCT 06-006', 'CHAR 08-002': 'CHARLES PRECINCT 08-002', 'CHAR 06-013': 'CHARLES PRECINCT 06-013', 'CHAR 07-003': 'CHARLES PRECINCT 07-003', 'CHAR 02-001': 'CHARLES PRECINCT 02-001', 'CHAR 06-010': 'CHARLES PRECINCT 06-010', 'CHAR 06-019': 'CHARLES PRECINCT 06-019', 'CHAR 07-002': 'CHARLES PRECINCT 07-002', 'CHAR 07-004': 'CHARLES PRECINCT 07-004', 'CHAR 06-004': 'CHARLES PRECINCT 06-004', 'CHAR 08-003': 'CHARLES PRECINCT 08-003', 'CHAR 06-007': 'CHARLES PRECINCT 06-007', 'CHAR 06-018': 'CHARLES PRECINCT 06-018', 'CHAR 06-011': 'CHARLES PRECINCT 06-011', 'CHAR 06-005': 'CHARLES PRECINCT 06-005', 'CHAR 07-001': 'CHARLES PRECINCT 07-001', 'CHAR 06-003': 'CHARLES PRECINCT 06-003', 'CHAR 06-016': 'CHARLES PRECINCT 06-016', 'CHAR 06-008': 'CHARLES PRECINCT 06-008', 'CHAR 06-009': 'CHARLES PRECINCT 06-009', 'CHAR 06-020': 'CHARLES PRECINCT 06-020', 'CHAR 06-017': 'CHARLES PRECINCT 06-017', 'CHAR 06-021': 'CHARLES PRECINCT 06-021', 'CHAR 06-015': 'CHARLES PRECINCT 06-015', 'CHAR 06-012': 'CHARLES PRECINCT 06-012', 'CHAR 06-022': 'CHARLES PRECINCT 06-022', 'CHAR 08-005': 'CHARLES PRECINCT 08-005', 'DORC 07-006': 'DORCHESTER PRECINCT 07-006', 'DORC 15-001': 'DORCHESTER PRECINCT 15-001', 'DORC 02-001': 'DORCHESTER PRECINCT 02-001', 'DORC 01-001': 'DORCHESTER PRECINCT 01-001', 'DORC 14-001': 'DORCHESTER PRECINCT 14-001', 'DORC 08-001': 'DORCHESTER PRECINCT 08-001', 'DORC 15-003': 'DORCHESTER PRECINCT 15-003', 'DORC 14-003': 'DORCHESTER PRECINCT 14-003', 'DORC 09-001': 'DORCHESTER PRECINCT 09-001', 'DORC 07-009': 'DORCHESTER PRECINCT 07-009', 'DORC 07-003': 'DORCHESTER PRECINCT 07-003', 'DORC 07-002': 'DORCHESTER PRECINCT 07-002', 'DORC 16-001': 'DORCHESTER PRECINCT 16-001', 'DORC 07-001': 'DORCHESTER PRECINCT 07-001', 'DORC 06-001': 'DORCHESTER PRECINCT 06-001', 'DORC 03-001': 'DORCHESTER PRECINCT 03-001', 'DORC 12-001': 'DORCHESTER PRECINCT 12-001', 'DORC 03-002': 'DORCHESTER PRECINCT 03-002', 'DORC 13-001': 'DORCHESTER PRECINCT 13-001', 'DORC 07-004': 'DORCHESTER PRECINCT 07-004', 'DORC 05-001': 'DORCHESTER PRECINCT 05-001', 'DORC 02-004': 'DORCHESTER PRECINCT 02-004', 'DORC 12-002': 'DORCHESTER PRECINCT 12-002', 'DORC 10-001': 'DORCHESTER PRECINCT 10-001', 'DORC 17-001': 'DORCHESTER PRECINCT 17-001', 'DORC 04-001': 'DORCHESTER PRECINCT 04-001', 'DORC 02-003': 'DORCHESTER PRECINCT 02-003', 'DORC 02-002': 'DORCHESTER PRECINCT 02-002', 'DORC 07-010': 'DORCHESTER PRECINCT 07-010', 'DORC 13-002': 'DORCHESTER PRECINCT 13-002', 'DORC 15-002': 'DORCHESTER PRECINCT 15-002', 'DORC 07-005': 'DORCHESTER PRECINCT 07-005', 'DORC 07-007': 'DORCHESTER PRECINCT 07-007', 'DORC 11-001': 'DORCHESTER PRECINCT 11-001', 'DORC 14-002': 'DORCHESTER PRECINCT 14-002', 'DORC 18-001': 'DORCHESTER PRECINCT 18-001', 'DORC 07-008': 'DORCHESTER PRECINCT 07-008', 'DORC 17-002': 'DORCHESTER PRECINCT 17-002', 'DORC 07-011': 'DORCHESTER PRECINCT 07-011', 'DORC 14-004': 'DORCHESTER PRECINCT 14-004', 'FRED 09-007': 'FREDERICK PRECINCT 09-007', 'FRED 18-001': 'FREDERICK PRECINCT 18-001', 'FRED 09-002': 'FREDERICK PRECINCT 09-002', 'FRED 14-001': 'FREDERICK PRECINCT 14-001', 'FRED 13-001': 'FREDERICK PRECINCT 13-001', 'FRED 23-001': 'FREDERICK PRECINCT 23-001', 'FRED 09-004': 'FREDERICK PRECINCT 09-004', 'FRED 16-001': 'FREDERICK PRECINCT 16-001', 'FRED 26-001': 'FREDERICK PRECINCT 26-001', 'FRED 05-001': 'FREDERICK PRECINCT 05-001', 'FRED 09-001': 'FREDERICK PRECINCT 09-001', 'FRED 01-001': 'FREDERICK PRECINCT 01-001', 'FRED 03-002': 'FREDERICK PRECINCT 03-002', 'FRED 07-002': 'FREDERICK PRECINCT 07-002', 'FRED 18-002': 'FREDERICK PRECINCT 18-002', 'FRED 26-002': 'FREDERICK PRECINCT 26-002', 'FRED 25-001': 'FREDERICK PRECINCT 25-001', 'FRED 09-006': 'FREDERICK PRECINCT 09-006', 'FRED 15-002': 'FREDERICK PRECINCT 15-002', 'FRED 12-001': 'FREDERICK PRECINCT 12-001', 'FRED 02-015': 'FREDERICK PRECINCT 02-015', 'FRED 15-001': 'FREDERICK PRECINCT 15-001', 'FRED 01-002': 'FREDERICK PRECINCT 01-002', 'FRED 02-010': 'FREDERICK PRECINCT 02-010', 'FRED 11-001': 'FREDERICK PRECINCT 11-001', 'FRED 02-005': 'FREDERICK PRECINCT 02-005', 'FRED 20-001': 'FREDERICK PRECINCT 20-001', 'FRED 08-001': 'FREDERICK PRECINCT 08-001', 'FRED 02-007': 'FREDERICK PRECINCT 02-007', 'FRED 07-003': 'FREDERICK PRECINCT 07-003', 'FRED 02-006': 'FREDERICK PRECINCT 02-006', 'FRED 06-001': 'FREDERICK PRECINCT 06-001', 'FRED 02-004': 'FREDERICK PRECINCT 02-004', 'FRED 17-001': 'FREDERICK PRECINCT 17-001', 'FRED 02-008': 'FREDERICK PRECINCT 02-008', 'FRED 03-003': 'FREDERICK PRECINCT 03-003', 'FRED 21-001': 'FREDERICK PRECINCT 21-001', 'FRED 21-004': 'FREDERICK PRECINCT 21-004', 'FRED 02-003': 'FREDERICK PRECINCT 02-003', 'FRED 01-003': 'FREDERICK PRECINCT 01-003', 'FRED 24-001': 'FREDERICK PRECINCT 24-001', 'FRED 03-001': 'FREDERICK PRECINCT 03-001', 'FRED 10-001': 'FREDERICK PRECINCT 10-001', 'FRED 19-001': 'FREDERICK PRECINCT 19-001', 'FRED 07-004': 'FREDERICK PRECINCT 07-004', 'FRED 18-003': 'FREDERICK PRECINCT 18-003', 'FRED 07-001': 'FREDERICK PRECINCT 07-001', 'FRED 09-005': 'FREDERICK PRECINCT 09-005', 'FRED 21-002': 'FREDERICK PRECINCT 21-002', 'FRED 02-018': 'FREDERICK PRECINCT 02-018', 'FRED 02-013': 'FREDERICK PRECINCT 02-013', 'FRED 02-014': 'FREDERICK PRECINCT 02-014', 'FRED 02-001': 'FREDERICK PRECINCT 02-001', 'FRED 22-001': 'FREDERICK PRECINCT 22-001', 'FRED 02-011': 'FREDERICK PRECINCT 02-011', 'FRED 09-008': 'FREDERICK PRECINCT 09-008', 'FRED 21-003': 'FREDERICK PRECINCT 21-003', 'FRED 02-016': 'FREDERICK PRECINCT 02-016', 'FRED 09-009': 'FREDERICK PRECINCT 09-009', 'FRED 04-001': 'FREDERICK PRECINCT 04-001', 'FRED 24-002': 'FREDERICK PRECINCT 24-002', 'FRED 02-017': 'FREDERICK PRECINCT 02-017', 'FRED 24-005': 'FREDERICK PRECINCT 24-005', 'FRED 02-002': 'FREDERICK PRECINCT 02-002', 'FRED 21-007': 'FREDERICK PRECINCT 21-007', 'FRED 24-003': 'FREDERICK PRECINCT 24-003', 'FRED 09-003': 'FREDERICK PRECINCT 09-003', 'FRED 14-002': 'FREDERICK PRECINCT 14-002', 'FRED 07-007': 'FREDERICK PRECINCT 07-007', 'FRED 13-003': 'FREDERICK PRECINCT 13-003', 'FRED 13-002': 'FREDERICK PRECINCT 13-002', 'FRED 07-005': 'FREDERICK PRECINCT 07-005', 'FRED 23-002': 'FREDERICK PRECINCT 23-002', 'FRED 24-006': 'FREDERICK PRECINCT 24-006', 'FRED 21-008': 'FREDERICK PRECINCT 21-008', 'FRED 22-002': 'FREDERICK PRECINCT 22-002', 'FRED 24-004': 'FREDERICK PRECINCT 24-004', 'FRED 07-006': 'FREDERICK PRECINCT 07-006', 'FRED 23-003': 'FREDERICK PRECINCT 23-003', 'FRED 20-002': 'FREDERICK PRECINCT 20-002', 'FRED 23-004': 'FREDERICK PRECINCT 23-004', 'FRED 12-002': 'FREDERICK PRECINCT 12-002', 'FRED 21-006': 'FREDERICK PRECINCT 21-006', 'FRED 21-005': 'FREDERICK PRECINCT 21-005', 'GARR 16-000': 'GARRETT PRECINCT 16-001', 'GARR 03-001': 'GARRETT PRECINCT 03-001', 'GARR 05-000': 'GARRETT PRECINCT 05-001', 'GARR 14-001': 'GARRETT PRECINCT 14-001', 'GARR 06-000': 'GARRETT PRECINCT 06-001', 'GARR 01-000': 'GARRETT PRECINCT 01-001', 'GARR 02-000': 'GARRETT PRECINCT 02-001', 'GARR 10-000': 'GARRETT PRECINCT 10-001', 'GARR 07-000': 'GARRETT PRECINCT 07-001', 'GARR 09-000': 'GARRETT PRECINCT 09-001', 'GARR 14-002': 'GARRETT PRECINCT 14-002', 'GARR 12-000': 'GARRETT PRECINCT 12-001', 'GARR 04-000': 'GARRETT PRECINCT 04-001', 'GARR 08-002': 'GARRETT PRECINCT 08-002', 'GARR 03-002': 'GARRETT PRECINCT 03-002', 'GARR 15-000': 'GARRETT PRECINCT 15-001', 'GARR 08-001': 'GARRETT PRECINCT 08-001', 'GARR 13-000': 'GARRETT PRECINCT 13-001', 'GARR 11-000': 'GARRETT PRECINCT 11-001', 'HARF 03-003': 'HARFORD PRECINCT 03-003', 'HARF 03-005': 'HARFORD PRECINCT 03-005', 'HARF 02-002': 'HARFORD PRECINCT 02-002', 'HARF 01-014': 'HARFORD PRECINCT 01-014', 'HARF 03-009': 'HARFORD PRECINCT 03-009', 'HARF 03-006': 'HARFORD PRECINCT 03-006', 'HARF 03-013': 'HARFORD PRECINCT 03-013', 'HARF 03-022': 'HARFORD PRECINCT 03-022', 'HARF 06-006': 'HARFORD PRECINCT 06-006', 'HARF 04-003': 'HARFORD PRECINCT 04-003', 'HARF 05-001': 'HARFORD PRECINCT 05-001', 'HARF 01-008': 'HARFORD PRECINCT 01-008', 'HARF 03-008': 'HARFORD PRECINCT 03-008', 'HARF 03-007': 'HARFORD PRECINCT 03-007', 'HARF 05-003': 'HARFORD PRECINCT 05-003', 'HARF 04-005': 'HARFORD PRECINCT 04-005', 'HARF 03-021': 'HARFORD PRECINCT 03-021', 'HARF 03-019': 'HARFORD PRECINCT 03-019', 'HARF 01-012': 'HARFORD PRECINCT 01-012', 'HARF 04-002': 'HARFORD PRECINCT 04-002', 'HARF 01-011': 'HARFORD PRECINCT 01-011', 'HARF 05-002': 'HARFORD PRECINCT 05-002', 'HARF 03-002': 'HARFORD PRECINCT 03-002', 'HARF 03-023': 'HARFORD PRECINCT 03-023', 'HARF 04-004': 'HARFORD PRECINCT 04-004', 'HARF 03-020': 'HARFORD PRECINCT 03-020', 'HARF 01-045': 'HARFORD PRECINCT 01-045', 'HARF 01-010': 'HARFORD PRECINCT 01-010', 'HARF 03-024': 'HARFORD PRECINCT 03-024', 'HARF 01-015': 'HARFORD PRECINCT 01-015', 'HARF 03-017': 'HARFORD PRECINCT 03-017', 'HARF 01-016': 'HARFORD PRECINCT 01-016', 'HARF 01-004': 'HARFORD PRECINCT 01-004', 'HARF 03-010': 'HARFORD PRECINCT 03-010', 'HARF 06-002': 'HARFORD PRECINCT 06-002', 'HARF 01-009': 'HARFORD PRECINCT 01-009', 'HARF 03-004': 'HARFORD PRECINCT 03-004', 'HARF 03-018': 'HARFORD PRECINCT 03-018', 'HARF 03-014': 'HARFORD PRECINCT 03-014', 'HARF 03-015': 'HARFORD PRECINCT 03-015', 'HARF 01-002': 'HARFORD PRECINCT 01-002', 'HARF 03-016': 'HARFORD PRECINCT 03-016', 'HARF 03-012': 'HARFORD PRECINCT 03-012', 'HARF 01-020': 'HARFORD PRECINCT 01-020', 'HARF 04-006': 'HARFORD PRECINCT 04-006', 'HARF 03-011': 'HARFORD PRECINCT 03-011', 'HARF 05-004': 'HARFORD PRECINCT 05-004', 'HARF 06-003': 'HARFORD PRECINCT 06-003', 'HARF 01-007': 'HARFORD PRECINCT 01-007', 'HARF 01-018': 'HARFORD PRECINCT 01-018', 'HARF 02-011': 'HARFORD PRECINCT 02-011', 'HARF 04-001': 'HARFORD PRECINCT 04-001', 'HARF 06-001': 'HARFORD PRECINCT 06-001', 'HARF 01-041': 'HARFORD PRECINCT 01-041', 'HARF 01-027': 'HARFORD PRECINCT 01-027', 'HARF 02-010': 'HARFORD PRECINCT 02-010', 'HARF 01-019': 'HARFORD PRECINCT 01-019', 'HARF 04-007': 'HARFORD PRECINCT 04-007', 'HARF 02-015': 'HARFORD PRECINCT 02-015', 'HARF 01-005': 'HARFORD PRECINCT 01-005', 'HARF 01-001': 'HARFORD PRECINCT 01-001', 'HARF 02-003': 'HARFORD PRECINCT 02-003', 'HARF 01-026': 'HARFORD PRECINCT 01-026', 'HARF 01-013': 'HARFORD PRECINCT 01-013', 'HARF 03-026': 'HARFORD PRECINCT 03-026', 'HARF 01-021': 'HARFORD PRECINCT 01-021', 'HARF 02-014': 'HARFORD PRECINCT 02-014', 'HARF 01-006': 'HARFORD PRECINCT 01-006', 'HARF 01-003': 'HARFORD PRECINCT 01-003', 'HARF 01-024': 'HARFORD PRECINCT 01-024', 'HARF 02-001': 'HARFORD PRECINCT 02-001', 'HARF 01-017': 'HARFORD PRECINCT 01-017', 'HARF 03-027': 'HARFORD PRECINCT 03-027', 'HARF 01-023': 'HARFORD PRECINCT 01-023', 'HARF 02-019': 'HARFORD PRECINCT 02-019', 'HARF 06-007': 'HARFORD PRECINCT 06-007', 'HARF 06-005': 'HARFORD PRECINCT 06-005', 'HARF 06-004': 'HARFORD PRECINCT 06-004', 'HARF 01-022': 'HARFORD PRECINCT 01-022', 'HARF 02-004': 'HARFORD PRECINCT 02-004', 'HARF 02-007': 'HARFORD PRECINCT 02-007', 'HARF 02-005': 'HARFORD PRECINCT 02-005', 'HARF 02-021': 'HARFORD PRECINCT 02-021', 'HARF 02-020': 'HARFORD PRECINCT 02-020', 'HARF 02-022': 'HARFORD PRECINCT 02-022', 'HARF 01-025': 'HARFORD PRECINCT 01-025', 'HARF 04-008': 'HARFORD PRECINCT 04-008', 'HARF 02-006': 'HARFORD PRECINCT 02-006', 'HARF 03-025': 'HARFORD PRECINCT 03-025', 'HOWA 05-006': 'HOWARD PRECINCT 05-006', 'HOWA 01-004': 'HOWARD PRECINCT 01-004', 'HOWA 03-002': 'HOWARD PRECINCT 03-002', 'HOWA 05-001': 'HOWARD PRECINCT 05-001', 'HOWA 04-006': 'HOWARD PRECINCT 04-006', 'HOWA 03-005': 'HOWARD PRECINCT 03-005', 'HOWA 05-019': 'HOWARD PRECINCT 05-019', 'HOWA 06-028': 'HOWARD PRECINCT 06-028', 'HOWA 04-004': 'HOWARD PRECINCT 04-004', 'HOWA 05-022': 'HOWARD PRECINCT 05-022', 'HOWA 06-029': 'HOWARD PRECINCT 06-029', 'HOWA 05-010': 'HOWARD PRECINCT 05-010', 'HOWA 04-003': 'HOWARD PRECINCT 04-003', 'HOWA 02-003': 'HOWARD PRECINCT 02-003', 'HOWA 01-001': 'HOWARD PRECINCT 01-001', 'HOWA 02-002': 'HOWARD PRECINCT 02-002', 'HOWA 05-005': 'HOWARD PRECINCT 05-005', 'HOWA 04-002': 'HOWARD PRECINCT 04-002', 'HOWA 04-005': 'HOWARD PRECINCT 04-005', 'HOWA 06-035': 'HOWARD PRECINCT 06-035', 'HOWA 05-020': 'HOWARD PRECINCT 05-020', 'HOWA 02-008': 'HOWARD PRECINCT 02-008', 'HOWA 02-016': 'HOWARD PRECINCT 02-016', 'HOWA 06-004': 'HOWARD PRECINCT 06-004', 'HOWA 04-001': 'HOWARD PRECINCT 04-001', 'HOWA 02-018': 'HOWARD PRECINCT 02-018', 'HOWA 03-003': 'HOWARD PRECINCT 03-003', 'HOWA 01-014': 'HOWARD PRECINCT 01-014', 'HOWA 03-006': 'HOWARD PRECINCT 03-006', 'HOWA 02-012': 'HOWARD PRECINCT 02-012', 'HOWA 02-007': 'HOWARD PRECINCT 02-007', 'HOWA 02-019': 'HOWARD PRECINCT 02-019', 'HOWA 01-015': 'HOWARD PRECINCT 01-015', 'HOWA 05-015': 'HOWARD PRECINCT 05-015', 'HOWA 06-032': 'HOWARD PRECINCT 06-032', 'HOWA 01-008': 'HOWARD PRECINCT 01-008', 'HOWA 03-001': 'HOWARD PRECINCT 03-001', 'HOWA 01-003': 'HOWARD PRECINCT 01-003', 'HOWA 06-026': 'HOWARD PRECINCT 06-026', 'HOWA 06-002': 'HOWARD PRECINCT 06-002', 'HOWA 01-013': 'HOWARD PRECINCT 01-013', 'HOWA 02-011': 'HOWARD PRECINCT 02-011', 'HOWA 05-011': 'HOWARD PRECINCT 05-011', 'HOWA 06-024': 'HOWARD PRECINCT 06-024', 'HOWA 02-015': 'HOWARD PRECINCT 02-015', 'HOWA 02-001': 'HOWARD PRECINCT 02-001', 'HOWA 01-009': 'HOWARD PRECINCT 01-009', 'HOWA 06-003': 'HOWARD PRECINCT 06-003', 'HOWA 06-033': 'HOWARD PRECINCT 06-033', 'HOWA 05-004': 'HOWARD PRECINCT 05-004', 'HOWA 06-001': 'HOWARD PRECINCT 06-001', 'HOWA 06-018': 'HOWARD PRECINCT 06-018', 'HOWA 05-003': 'HOWARD PRECINCT 05-003', 'HOWA 06-027': 'HOWARD PRECINCT 06-027', 'HOWA 02-006': 'HOWARD PRECINCT 02-006', 'HOWA 02-026': 'HOWARD PRECINCT 02-026', 'HOWA 01-006': 'HOWARD PRECINCT 01-006', 'HOWA 06-011': 'HOWARD PRECINCT 06-011', 'HOWA 02-025': 'HOWARD PRECINCT 02-025', 'HOWA 06-016': 'HOWARD PRECINCT 06-016', 'HOWA 02-024': 'HOWARD PRECINCT 02-024', 'HOWA 02-009': 'HOWARD PRECINCT 02-009', 'HOWA 06-020': 'HOWARD PRECINCT 06-020', 'HOWA 01-021': 'HOWARD PRECINCT 01-021', 'HOWA 02-005': 'HOWARD PRECINCT 02-005', 'HOWA 06-010': 'HOWARD PRECINCT 06-010', 'HOWA 01-010': 'HOWARD PRECINCT 01-010', 'HOWA 06-015': 'HOWARD PRECINCT 06-015', 'HOWA 02-004': 'HOWARD PRECINCT 02-004', 'HOWA 05-017': 'HOWARD PRECINCT 05-017', 'HOWA 02-010': 'HOWARD PRECINCT 02-010', 'HOWA 05-012': 'HOWARD PRECINCT 05-012', 'HOWA 05-016': 'HOWARD PRECINCT 05-016', 'HOWA 06-019': 'HOWARD PRECINCT 06-019', 'HOWA 06-017': 'HOWARD PRECINCT 06-017', 'HOWA 06-006': 'HOWARD PRECINCT 06-006', 'HOWA 06-005': 'HOWARD PRECINCT 06-005', 'HOWA 05-024': 'HOWARD PRECINCT 05-024', 'HOWA 06-014': 'HOWARD PRECINCT 06-014', 'HOWA 01-011': 'HOWARD PRECINCT 01-011', 'HOWA 06-025': 'HOWARD PRECINCT 06-025', 'HOWA 06-013': 'HOWARD PRECINCT 06-013', 'HOWA 05-021': 'HOWARD PRECINCT 05-021', 'HOWA 06-022': 'HOWARD PRECINCT 06-022', 'HOWA 01-012': 'HOWARD PRECINCT 01-012', 'HOWA 05-009': 'HOWARD PRECINCT 05-009', 'HOWA 06-009': 'HOWARD PRECINCT 06-009', 'HOWA 06-034': 'HOWARD PRECINCT 06-034', 'HOWA 05-008': 'HOWARD PRECINCT 05-008', 'HOWA 05-013': 'HOWARD PRECINCT 05-013', 'HOWA 05-007': 'HOWARD PRECINCT 05-007', 'HOWA 06-023': 'HOWARD PRECINCT 06-023', 'HOWA 06-031': 'HOWARD PRECINCT 06-031', 'HOWA 06-008': 'HOWARD PRECINCT 06-008', 'HOWA 02-023': 'HOWARD PRECINCT 02-023', 'HOWA 05-023': 'HOWARD PRECINCT 05-023', 'HOWA 01-005': 'HOWARD PRECINCT 01-005', 'HOWA 02-014': 'HOWARD PRECINCT 02-014', 'HOWA 02-020': 'HOWARD PRECINCT 02-020', 'HOWA 02-013': 'HOWARD PRECINCT 02-013', 'HOWA 01-016': 'HOWARD PRECINCT 01-016', 'HOWA 01-002': 'HOWARD PRECINCT 01-002', 'HOWA 06-007': 'HOWARD PRECINCT 06-007', 'HOWA 01-020': 'HOWARD PRECINCT 01-020', 'HOWA 05-018': 'HOWARD PRECINCT 05-018', 'HOWA 05-014': 'HOWARD PRECINCT 05-014', 'HOWA 05-002': 'HOWARD PRECINCT 05-002', 'HOWA 06-012': 'HOWARD PRECINCT 06-012', 'HOWA 01-007': 'HOWARD PRECINCT 01-007', 'HOWA 06-030': 'HOWARD PRECINCT 06-030', 'HOWA 06-021': 'HOWARD PRECINCT 06-021', 'HOWA 01-018': 'HOWARD PRECINCT 01-018', 'HOWA 02-017': 'HOWARD PRECINCT 02-017', 'HOWA 01-017': 'HOWARD PRECINCT 01-017', 'HOWA 02-021': 'HOWARD PRECINCT 02-021', 'HOWA 03-004': 'HOWARD PRECINCT 03-004', 'HOWA 01-019': 'HOWARD PRECINCT 01-019', 'HOWA 02-022': 'HOWARD PRECINCT 02-022', 'KENT 05-001': 'KENT PRECINCT 05-001', 'KENT 01-002': 'KENT PRECINCT 01-002', 'KENT 02-001': 'KENT PRECINCT 02-001', 'KENT 03-001': 'KENT PRECINCT 03-001', 'KENT 01-001': 'KENT PRECINCT 01-001', 'KENT 06-001': 'KENT PRECINCT 06-001', 'KENT 04-001': 'KENT PRECINCT 04-001', 'KENT 04-002': 'KENT PRECINCT 04-002', 'KENT 03-002': 'KENT PRECINCT 03-002', 'KENT 07-001': 'KENT PRECINCT 07-001', 'MONT 13-038': 'MONTGOMERY PRECINCT 13-038', 'MONT 12-003': 'MONTGOMERY PRECINCT 12-003', 'MONT 09-035': 'MONTGOMERY PRECINCT 09-035', 'MONT 03-001': 'MONTGOMERY PRECINCT 03-001', 'MONT 08-002': 'MONTGOMERY PRECINCT 08-002', 'MONT 08-006': 'MONTGOMERY PRECINCT 08-006', 'MONT 06-005': 'MONTGOMERY PRECINCT 06-005', 'MONT 08-003': 'MONTGOMERY PRECINCT 08-003', 'MONT 02-006': 'MONTGOMERY PRECINCT 02-006', 'MONT 07-023': 'MONTGOMERY PRECINCT 07-023', 'MONT 03-002': 'MONTGOMERY PRECINCT 03-002', 'MONT 08-007': 'MONTGOMERY PRECINCT 08-007', 'MONT 07-019': 'MONTGOMERY PRECINCT 07-019', 'MONT 08-005': 'MONTGOMERY PRECINCT 08-005', 'MONT 04-020': 'MONTGOMERY PRECINCT 04-020', 'MONT 09-027': 'MONTGOMERY PRECINCT 09-027', 'MONT 04-016': 'MONTGOMERY PRECINCT 04-016', 'MONT 08-008': 'MONTGOMERY PRECINCT 08-008', 'MONT 08-001': 'MONTGOMERY PRECINCT 08-001', 'MONT 08-009': 'MONTGOMERY PRECINCT 08-009', 'MONT 06-009': 'MONTGOMERY PRECINCT 06-009', 'MONT 07-015': 'MONTGOMERY PRECINCT 07-015', 'MONT 02-008': 'MONTGOMERY PRECINCT 02-008', 'MONT 06-002': 'MONTGOMERY PRECINCT 06-002', 'MONT 01-005': 'MONTGOMERY PRECINCT 01-005', 'MONT 02-001': 'MONTGOMERY PRECINCT 02-001', 'MONT 08-012': 'MONTGOMERY PRECINCT 08-012', 'MONT 07-013': 'MONTGOMERY PRECINCT 07-013', 'MONT 06-015': 'MONTGOMERY PRECINCT 06-015', 'MONT 10-010': 'MONTGOMERY PRECINCT 10-010', 'MONT 12-002': 'MONTGOMERY PRECINCT 12-002', 'MONT 02-011': 'MONTGOMERY PRECINCT 02-011', 'MONT 06-007': 'MONTGOMERY PRECINCT 06-007', 'MONT 05-009': 'MONTGOMERY PRECINCT 05-009', 'MONT 07-004': 'MONTGOMERY PRECINCT 07-004', 'MONT 10-009': 'MONTGOMERY PRECINCT 10-009', 'MONT 12-001': 'MONTGOMERY PRECINCT 12-001', 'MONT 04-008': 'MONTGOMERY PRECINCT 04-008', 'MONT 04-018': 'MONTGOMERY PRECINCT 04-018', 'MONT 13-061': 'MONTGOMERY PRECINCT 13-061', 'MONT 01-001': 'MONTGOMERY PRECINCT 01-001', 'MONT 09-013': 'MONTGOMERY PRECINCT 09-013', 'MONT 01-003': 'MONTGOMERY PRECINCT 01-003', 'MONT 13-051': 'MONTGOMERY PRECINCT 13-051', 'MONT 06-006': 'MONTGOMERY PRECINCT 06-006', 'MONT 02-004': 'MONTGOMERY PRECINCT 02-004', 'MONT 04-012': 'MONTGOMERY PRECINCT 04-012', 'MONT 06-001': 'MONTGOMERY PRECINCT 06-001', 'MONT 12-005': 'MONTGOMERY PRECINCT 12-005', 'MONT 10-002': 'MONTGOMERY PRECINCT 10-002', 'MONT 13-002': 'MONTGOMERY PRECINCT 13-002', 'MONT 13-020': 'MONTGOMERY PRECINCT 13-020', 'MONT 01-002': 'MONTGOMERY PRECINCT 01-002', 'MONT 07-024': 'MONTGOMERY PRECINCT 07-024', 'MONT 10-007': 'MONTGOMERY PRECINCT 10-007', 'MONT 04-003': 'MONTGOMERY PRECINCT 04-003', 'MONT 05-012': 'MONTGOMERY PRECINCT 05-012', 'MONT 12-004': 'MONTGOMERY PRECINCT 12-004', 'MONT 05-016': 'MONTGOMERY PRECINCT 05-016', 'MONT 06-003': 'MONTGOMERY PRECINCT 06-003', 'MONT 04-002': 'MONTGOMERY PRECINCT 04-002', 'MONT 13-023': 'MONTGOMERY PRECINCT 13-023', 'MONT 04-013': 'MONTGOMERY PRECINCT 04-013', 'MONT 07-018': 'MONTGOMERY PRECINCT 07-018', 'MONT 07-025': 'MONTGOMERY PRECINCT 07-025', 'MONT 10-003': 'MONTGOMERY PRECINCT 10-003', 'MONT 08-011': 'MONTGOMERY PRECINCT 08-011', 'MONT 07-020': 'MONTGOMERY PRECINCT 07-020', 'MONT 04-010': 'MONTGOMERY PRECINCT 04-010', 'MONT 10-012': 'MONTGOMERY PRECINCT 10-012', 'MONT 02-005': 'MONTGOMERY PRECINCT 02-005', 'MONT 04-014': 'MONTGOMERY PRECINCT 04-014', 'MONT 04-021': 'MONTGOMERY PRECINCT 04-021', 'MONT 11-000': 'MONTGOMERY PRECINCT 11-000', 'MONT 07-003': 'MONTGOMERY PRECINCT 07-003', 'MONT 09-022': 'MONTGOMERY PRECINCT 09-022', 'MONT 09-014': 'MONTGOMERY PRECINCT 09-014', 'MONT 05-015': 'MONTGOMERY PRECINCT 05-015', 'MONT 04-024': 'MONTGOMERY PRECINCT 04-024', 'MONT 07-026': 'MONTGOMERY PRECINCT 07-026', 'MONT 07-012': 'MONTGOMERY PRECINCT 07-012', 'MONT 07-010': 'MONTGOMERY PRECINCT 07-010', 'MONT 13-069': 'MONTGOMERY PRECINCT 13-069', 'MONT 13-034': 'MONTGOMERY PRECINCT 13-034', 'MONT 13-046': 'MONTGOMERY PRECINCT 13-046', 'MONT 02-002': 'MONTGOMERY PRECINCT 02-002', 'MONT 13-054': 'MONTGOMERY PRECINCT 13-054', 'MONT 06-004': 'MONTGOMERY PRECINCT 06-004', 'MONT 05-001': 'MONTGOMERY PRECINCT 05-001', 'MONT 07-002': 'MONTGOMERY PRECINCT 07-002', 'MONT 09-006': 'MONTGOMERY PRECINCT 09-006', 'MONT 05-003': 'MONTGOMERY PRECINCT 05-003', 'MONT 09-002': 'MONTGOMERY PRECINCT 09-002', 'MONT 13-052': 'MONTGOMERY PRECINCT 13-052', 'MONT 09-012': 'MONTGOMERY PRECINCT 09-012', 'MONT 13-058': 'MONTGOMERY PRECINCT 13-058', 'MONT 07-011': 'MONTGOMERY PRECINCT 07-011', 'MONT 07-027': 'MONTGOMERY PRECINCT 07-027', 'MONT 04-006': 'MONTGOMERY PRECINCT 04-006', 'MONT 05-019': 'MONTGOMERY PRECINCT 05-019', 'MONT 07-009': 'MONTGOMERY PRECINCT 07-009', 'MONT 04-005': 'MONTGOMERY PRECINCT 04-005', 'MONT 04-017': 'MONTGOMERY PRECINCT 04-017', 'MONT 04-007': 'MONTGOMERY PRECINCT 04-007', 'MONT 10-006': 'MONTGOMERY PRECINCT 10-006', 'MONT 04-034': 'MONTGOMERY PRECINCT 04-034', 'MONT 07-008': 'MONTGOMERY PRECINCT 07-008', 'MONT 09-004': 'MONTGOMERY PRECINCT 09-004', 'MONT 05-006': 'MONTGOMERY PRECINCT 05-006', 'MONT 04-031': 'MONTGOMERY PRECINCT 04-031', 'MONT 10-005': 'MONTGOMERY PRECINCT 10-005', 'MONT 04-001': 'MONTGOMERY PRECINCT 04-001', 'MONT 09-009': 'MONTGOMERY PRECINCT 09-009', 'MONT 13-071': 'MONTGOMERY PRECINCT 13-071', 'MONT 09-030': 'MONTGOMERY PRECINCT 09-030', 'MONT 13-003': 'MONTGOMERY PRECINCT 13-003', 'MONT 06-008': 'MONTGOMERY PRECINCT 06-008', 'MONT 07-005': 'MONTGOMERY PRECINCT 07-005', 'MONT 09-029': 'MONTGOMERY PRECINCT 09-029', 'MONT 05-008': 'MONTGOMERY PRECINCT 05-008', 'MONT 13-040': 'MONTGOMERY PRECINCT 13-040', 'MONT 09-031': 'MONTGOMERY PRECINCT 09-031', 'MONT 07-028': 'MONTGOMERY PRECINCT 07-028', 'MONT 07-022': 'MONTGOMERY PRECINCT 07-022', 'MONT 06-010': 'MONTGOMERY PRECINCT 06-010', 'MONT 02-003': 'MONTGOMERY PRECINCT 02-003', 'MONT 09-011': 'MONTGOMERY PRECINCT 09-011', 'MONT 09-025': 'MONTGOMERY PRECINCT 09-025', 'MONT 09-008': 'MONTGOMERY PRECINCT 09-008', 'MONT 04-030': 'MONTGOMERY PRECINCT 04-030', 'MONT 13-011': 'MONTGOMERY PRECINCT 13-011', 'MONT 09-021': 'MONTGOMERY PRECINCT 09-021', 'MONT 09-005': 'MONTGOMERY PRECINCT 09-005', 'MONT 09-001': 'MONTGOMERY PRECINCT 09-001', 'MONT 06-013': 'MONTGOMERY PRECINCT 06-013', 'MONT 05-004': 'MONTGOMERY PRECINCT 05-004', 'MONT 08-004': 'MONTGOMERY PRECINCT 08-004', 'MONT 07-016': 'MONTGOMERY PRECINCT 07-016', 'MONT 13-033': 'MONTGOMERY PRECINCT 13-033', 'MONT 05-023': 'MONTGOMERY PRECINCT 05-023', 'MONT 07-007': 'MONTGOMERY PRECINCT 07-007', 'MONT 04-004': 'MONTGOMERY PRECINCT 04-004', 'MONT 04-026': 'MONTGOMERY PRECINCT 04-026', 'MONT 01-006': 'MONTGOMERY PRECINCT 01-006', 'MONT 13-027': 'MONTGOMERY PRECINCT 13-027', 'MONT 13-056': 'MONTGOMERY PRECINCT 13-056', 'MONT 08-010': 'MONTGOMERY PRECINCT 08-010', 'MONT 13-024': 'MONTGOMERY PRECINCT 13-024', 'MONT 07-006': 'MONTGOMERY PRECINCT 07-006', 'MONT 13-015': 'MONTGOMERY PRECINCT 13-015', 'MONT 13-044': 'MONTGOMERY PRECINCT 13-044', 'MONT 05-017': 'MONTGOMERY PRECINCT 05-017', 'MONT 13-019': 'MONTGOMERY PRECINCT 13-019', 'MONT 05-018': 'MONTGOMERY PRECINCT 05-018', 'MONT 09-036': 'MONTGOMERY PRECINCT 09-036', 'MONT 10-001': 'MONTGOMERY PRECINCT 10-001', 'MONT 01-004': 'MONTGOMERY PRECINCT 01-004', 'MONT 04-025': 'MONTGOMERY PRECINCT 04-025', 'MONT 09-034': 'MONTGOMERY PRECINCT 09-034', 'MONT 13-025': 'MONTGOMERY PRECINCT 13-025', 'MONT 13-035': 'MONTGOMERY PRECINCT 13-035', 'MONT 09-037': 'MONTGOMERY PRECINCT 09-037', 'MONT 13-032': 'MONTGOMERY PRECINCT 13-032', 'MONT 13-007': 'MONTGOMERY PRECINCT 13-007', 'MONT 05-011': 'MONTGOMERY PRECINCT 05-011', 'MONT 04-019': 'MONTGOMERY PRECINCT 04-019', 'MONT 10-004': 'MONTGOMERY PRECINCT 10-004', 'MONT 05-005': 'MONTGOMERY PRECINCT 05-005', 'MONT 09-018': 'MONTGOMERY PRECINCT 09-018', 'MONT 13-014': 'MONTGOMERY PRECINCT 13-014', 'MONT 07-032': 'MONTGOMERY PRECINCT 07-032', 'MONT 13-036': 'MONTGOMERY PRECINCT 13-036', 'MONT 09-040': 'MONTGOMERY PRECINCT 09-040', 'MONT 10-013': 'MONTGOMERY PRECINCT 10-013', 'MONT 09-026': 'MONTGOMERY PRECINCT 09-026', 'MONT 04-032': 'MONTGOMERY PRECINCT 04-032', 'MONT 13-039': 'MONTGOMERY PRECINCT 13-039', 'MONT 13-043': 'MONTGOMERY PRECINCT 13-043', 'MONT 13-001': 'MONTGOMERY PRECINCT 13-001', 'MONT 09-028': 'MONTGOMERY PRECINCT 09-028', 'MONT 04-009': 'MONTGOMERY PRECINCT 04-009', 'MONT 13-031': 'MONTGOMERY PRECINCT 13-031', 'MONT 07-021': 'MONTGOMERY PRECINCT 07-021', 'MONT 09-007': 'MONTGOMERY PRECINCT 09-007', 'MONT 09-020': 'MONTGOMERY PRECINCT 09-020', 'MONT 13-029': 'MONTGOMERY PRECINCT 13-029', 'MONT 09-003': 'MONTGOMERY PRECINCT 09-003', 'MONT 13-059': 'MONTGOMERY PRECINCT 13-059', 'MONT 13-045': 'MONTGOMERY PRECINCT 13-045', 'MONT 13-053': 'MONTGOMERY PRECINCT 13-053', 'MONT 09-033': 'MONTGOMERY PRECINCT 09-033', 'MONT 09-024': 'MONTGOMERY PRECINCT 09-024', 'MONT 10-011': 'MONTGOMERY PRECINCT 10-011', 'MONT 13-037': 'MONTGOMERY PRECINCT 13-037', 'MONT 09-015': 'MONTGOMERY PRECINCT 09-015', 'MONT 13-030': 'MONTGOMERY PRECINCT 13-030', 'MONT 13-047': 'MONTGOMERY PRECINCT 13-047', 'MONT 07-001': 'MONTGOMERY PRECINCT 07-001', 'MONT 07-031': 'MONTGOMERY PRECINCT 07-031', 'MONT 06-014': 'MONTGOMERY PRECINCT 06-014', 'MONT 09-041': 'MONTGOMERY PRECINCT 09-041', 'MONT 13-005': 'MONTGOMERY PRECINCT 13-005', 'MONT 02-010': 'MONTGOMERY PRECINCT 02-010', 'MONT 04-029': 'MONTGOMERY PRECINCT 04-029', 'MONT 13-018': 'MONTGOMERY PRECINCT 13-018', 'MONT 13-064': 'MONTGOMERY PRECINCT 13-064', 'MONT 05-002': 'MONTGOMERY PRECINCT 05-002', 'MONT 05-021': 'MONTGOMERY PRECINCT 05-021', 'MONT 13-013': 'MONTGOMERY PRECINCT 13-013', 'MONT 05-013': 'MONTGOMERY PRECINCT 05-013', 'MONT 13-028': 'MONTGOMERY PRECINCT 13-028', 'MONT 05-022': 'MONTGOMERY PRECINCT 05-022', 'MONT 09-016': 'MONTGOMERY PRECINCT 09-016', 'MONT 04-015': 'MONTGOMERY PRECINCT 04-015', 'MONT 13-004': 'MONTGOMERY PRECINCT 13-004', 'MONT 13-050': 'MONTGOMERY PRECINCT 13-050', 'MONT 13-016': 'MONTGOMERY PRECINCT 13-016', 'MONT 13-021': 'MONTGOMERY PRECINCT 13-021', 'MONT 05-010': 'MONTGOMERY PRECINCT 05-010', 'MONT 13-042': 'MONTGOMERY PRECINCT 13-042', 'MONT 04-028': 'MONTGOMERY PRECINCT 04-028', 'MONT 13-008': 'MONTGOMERY PRECINCT 13-008', 'MONT 13-022': 'MONTGOMERY PRECINCT 13-022', 'MONT 13-063': 'MONTGOMERY PRECINCT 13-063', 'MONT 13-006': 'MONTGOMERY PRECINCT 13-006', 'MONT 05-014': 'MONTGOMERY PRECINCT 05-014', 'MONT 13-068': 'MONTGOMERY PRECINCT 13-068', 'MONT 13-049': 'MONTGOMERY PRECINCT 13-049', 'MONT 13-057': 'MONTGOMERY PRECINCT 13-057', 'MONT 13-010': 'MONTGOMERY PRECINCT 13-010', 'MONT 13-055': 'MONTGOMERY PRECINCT 13-055', 'MONT 01-007': 'MONTGOMERY PRECINCT 01-007', 'MONT 04-011': 'MONTGOMERY PRECINCT 04-011', 'MONT 09-010': 'MONTGOMERY PRECINCT 09-010', 'MONT 02-009': 'MONTGOMERY PRECINCT 02-009', 'MONT 13-067': 'MONTGOMERY PRECINCT 13-067', 'MONT 13-065': 'MONTGOMERY PRECINCT 13-065', 'MONT 09-038': 'MONTGOMERY PRECINCT 09-038', 'MONT 04-035': 'MONTGOMERY PRECINCT 04-035', 'MONT 04-023': 'MONTGOMERY PRECINCT 04-023', 'MONT 08-015': 'MONTGOMERY PRECINCT 08-015', 'MONT 09-032': 'MONTGOMERY PRECINCT 09-032', 'MONT 04-027': 'MONTGOMERY PRECINCT 04-027', 'MONT 02-007': 'MONTGOMERY PRECINCT 02-007', 'MONT 04-037': 'MONTGOMERY PRECINCT 04-037', 'MONT 04-033': 'MONTGOMERY PRECINCT 04-033', 'MONT 13-070': 'MONTGOMERY PRECINCT 13-070', 'MONT 06-011': 'MONTGOMERY PRECINCT 06-011', 'MONT 04-040': 'MONTGOMERY PRECINCT 04-040', 'MONT 04-038': 'MONTGOMERY PRECINCT 04-038', 'MONT 09-039': 'MONTGOMERY PRECINCT 09-039', 'MONT 13-060': 'MONTGOMERY PRECINCT 13-060', 'MONT 04-043': 'MONTGOMERY PRECINCT 04-043', 'MONT 08-014': 'MONTGOMERY PRECINCT 08-014', 'MONT 04-042': 'MONTGOMERY PRECINCT 04-042', 'MONT 04-036': 'MONTGOMERY PRECINCT 04-036', 'MONT 04-041': 'MONTGOMERY PRECINCT 04-041', 'MONT 06-016': 'MONTGOMERY PRECINCT 06-016', 'PRIN 10-005': "PRINCE GEORGE'S PRECINCT 10-005", 'PRIN 07-004': "PRINCE GEORGE'S PRECINCT 07-004", 'PRIN 07-005': "PRINCE GEORGE'S PRECINCT 07-005", 'PRIN 07-003': "PRINCE GEORGE'S PRECINCT 07-003", 'PRIN 15-005': "PRINCE GEORGE'S PRECINCT 15-005", 'PRIN 14-003': "PRINCE GEORGE'S PRECINCT 14-003", 'PRIN 16-003': "PRINCE GEORGE'S PRECINCT 16-003", 'PRIN 14-004': "PRINCE GEORGE'S PRECINCT 14-004", 'PRIN 01-005': "PRINCE GEORGE'S PRECINCT 01-005", 'PRIN 04-001': "PRINCE GEORGE'S PRECINCT 04-001", 'PRIN 05-001': "PRINCE GEORGE'S PRECINCT 05-001", 'PRIN 10-002': "PRINCE GEORGE'S PRECINCT 10-002", 'PRIN 01-001': "PRINCE GEORGE'S PRECINCT 01-001", 'PRIN 10-013': "PRINCE GEORGE'S PRECINCT 10-013", 'PRIN 21-007': "PRINCE GEORGE'S PRECINCT 21-007", 'PRIN 11-004': "PRINCE GEORGE'S PRECINCT 11-004", 'PRIN 14-009': "PRINCE GEORGE'S PRECINCT 14-009", 'PRIN 21-006': "PRINCE GEORGE'S PRECINCT 21-006", 'PRIN 14-006': "PRINCE GEORGE'S PRECINCT 14-006", 'PRIN 10-012': "PRINCE GEORGE'S PRECINCT 10-012", 'PRIN 21-003': "PRINCE GEORGE'S PRECINCT 21-003", 'PRIN 12-010': "PRINCE GEORGE'S PRECINCT 12-010", 'PRIN 15-001': "PRINCE GEORGE'S PRECINCT 15-001", 'PRIN 14-005': "PRINCE GEORGE'S PRECINCT 14-005", 'PRIN 03-003': "PRINCE GEORGE'S PRECINCT 03-003", 'PRIN 14-001': "PRINCE GEORGE'S PRECINCT 14-001", 'PRIN 10-006': "PRINCE GEORGE'S PRECINCT 10-006", 'PRIN 07-002': "PRINCE GEORGE'S PRECINCT 07-002", 'PRIN 07-010': "PRINCE GEORGE'S PRECINCT 07-010", 'PRIN 09-008': "PRINCE GEORGE'S PRECINCT 09-008", 'PRIN 07-007': "PRINCE GEORGE'S PRECINCT 07-007", 'PRIN 07-022': "PRINCE GEORGE'S PRECINCT 07-022", 'PRIN 11-003': "PRINCE GEORGE'S PRECINCT 11-003", 'PRIN 05-005': "PRINCE GEORGE'S PRECINCT 05-005", 'PRIN 07-006': "PRINCE GEORGE'S PRECINCT 07-006", 'PRIN 15-004': "PRINCE GEORGE'S PRECINCT 15-004", 'PRIN 01-004': "PRINCE GEORGE'S PRECINCT 01-004", 'PRIN 21-010': "PRINCE GEORGE'S PRECINCT 21-010", 'PRIN 05-007': "PRINCE GEORGE'S PRECINCT 05-007", 'PRIN 14-012': "PRINCE GEORGE'S PRECINCT 14-012", 'PRIN 07-009': "PRINCE GEORGE'S PRECINCT 07-009", 'PRIN 01-007': "PRINCE GEORGE'S PRECINCT 01-007", 'PRIN 02-007': "PRINCE GEORGE'S PRECINCT 02-007", 'PRIN 09-011': "PRINCE GEORGE'S PRECINCT 09-011", 'PRIN 09-001': "PRINCE GEORGE'S PRECINCT 09-001", 'PRIN 11-002': "PRINCE GEORGE'S PRECINCT 11-002", 'PRIN 09-006': "PRINCE GEORGE'S PRECINCT 09-006", 'PRIN 02-003': "PRINCE GEORGE'S PRECINCT 02-003", 'PRIN 19-001': "PRINCE GEORGE'S PRECINCT 19-001", 'PRIN 07-014': "PRINCE GEORGE'S PRECINCT 07-014", 'PRIN 21-001': "PRINCE GEORGE'S PRECINCT 21-001", 'PRIN 11-001': "PRINCE GEORGE'S PRECINCT 11-001", 'PRIN 19-003': "PRINCE GEORGE'S PRECINCT 19-003", 'PRIN 07-011': "PRINCE GEORGE'S PRECINCT 07-011", 'PRIN 14-007': "PRINCE GEORGE'S PRECINCT 14-007", 'PRIN 13-015': "PRINCE GEORGE'S PRECINCT 13-015", 'PRIN 13-010': "PRINCE GEORGE'S PRECINCT 13-010", 'PRIN 12-014': "PRINCE GEORGE'S PRECINCT 12-014", 'PRIN 15-006': "PRINCE GEORGE'S PRECINCT 15-006", 'PRIN 10-003': "PRINCE GEORGE'S PRECINCT 10-003", 'PRIN 16-004': "PRINCE GEORGE'S PRECINCT 16-004", 'PRIN 07-008': "PRINCE GEORGE'S PRECINCT 07-008", 'PRIN 05-012': "PRINCE GEORGE'S PRECINCT 05-012", 'PRIN 03-002': "PRINCE GEORGE'S PRECINCT 03-002", 'PRIN 13-014': "PRINCE GEORGE'S PRECINCT 13-014", 'PRIN 21-004': "PRINCE GEORGE'S PRECINCT 21-004", 'PRIN 01-003': "PRINCE GEORGE'S PRECINCT 01-003", 'PRIN 09-007': "PRINCE GEORGE'S PRECINCT 09-007", 'PRIN 03-006': "PRINCE GEORGE'S PRECINCT 03-006", 'PRIN 13-005': "PRINCE GEORGE'S PRECINCT 13-005", 'PRIN 12-005': "PRINCE GEORGE'S PRECINCT 12-005", 'PRIN 05-002': "PRINCE GEORGE'S PRECINCT 05-002", 'PRIN 08-001': "PRINCE GEORGE'S PRECINCT 08-001", 'PRIN 12-011': "PRINCE GEORGE'S PRECINCT 12-011", 'PRIN 02-008': "PRINCE GEORGE'S PRECINCT 02-008", 'PRIN 06-013': "PRINCE GEORGE'S PRECINCT 06-013", 'PRIN 05-006': "PRINCE GEORGE'S PRECINCT 05-006", 'PRIN 05-008': "PRINCE GEORGE'S PRECINCT 05-008", 'PRIN 21-002': "PRINCE GEORGE'S PRECINCT 21-002", 'PRIN 13-007': "PRINCE GEORGE'S PRECINCT 13-007", 'PRIN 14-002': "PRINCE GEORGE'S PRECINCT 14-002", 'PRIN 20-001': "PRINCE GEORGE'S PRECINCT 20-001", 'PRIN 01-002': "PRINCE GEORGE'S PRECINCT 01-002", 'PRIN 10-004': "PRINCE GEORGE'S PRECINCT 10-004", 'PRIN 21-018': "PRINCE GEORGE'S PRECINCT 21-018", 'PRIN 13-009': "PRINCE GEORGE'S PRECINCT 13-009", 'PRIN 06-007': "PRINCE GEORGE'S PRECINCT 06-007", 'PRIN 21-013': "PRINCE GEORGE'S PRECINCT 21-013", 'PRIN 21-011': "PRINCE GEORGE'S PRECINCT 21-011", 'PRIN 06-003': "PRINCE GEORGE'S PRECINCT 06-003", 'PRIN 05-003': "PRINCE GEORGE'S PRECINCT 05-003", 'PRIN 10-007': "PRINCE GEORGE'S PRECINCT 10-007", 'PRIN 21-014': "PRINCE GEORGE'S PRECINCT 21-014", 'PRIN 06-001': "PRINCE GEORGE'S PRECINCT 06-001", 'PRIN 02-006': "PRINCE GEORGE'S PRECINCT 02-006", 'PRIN 05-004': "PRINCE GEORGE'S PRECINCT 05-004", 'PRIN 13-011': "PRINCE GEORGE'S PRECINCT 13-011", 'PRIN 10-009': "PRINCE GEORGE'S PRECINCT 10-009", 'PRIN 17-001': "PRINCE GEORGE'S PRECINCT 17-001", 'PRIN 07-013': "PRINCE GEORGE'S PRECINCT 07-013", 'PRIN 09-013': "PRINCE GEORGE'S PRECINCT 09-013", 'PRIN 09-004': "PRINCE GEORGE'S PRECINCT 09-004", 'PRIN 07-016': "PRINCE GEORGE'S PRECINCT 07-016", 'PRIN 15-002': "PRINCE GEORGE'S PRECINCT 15-002", 'PRIN 13-006': "PRINCE GEORGE'S PRECINCT 13-006", 'PRIN 03-004': "PRINCE GEORGE'S PRECINCT 03-004", 'PRIN 17-010': "PRINCE GEORGE'S PRECINCT 17-010", 'PRIN 09-010': "PRINCE GEORGE'S PRECINCT 09-010", 'PRIN 14-010': "PRINCE GEORGE'S PRECINCT 14-010", 'PRIN 13-012': "PRINCE GEORGE'S PRECINCT 13-012", 'PRIN 12-015': "PRINCE GEORGE'S PRECINCT 12-015", 'PRIN 02-004': "PRINCE GEORGE'S PRECINCT 02-004", 'PRIN 10-008': "PRINCE GEORGE'S PRECINCT 10-008", 'PRIN 06-020': "PRINCE GEORGE'S PRECINCT 06-020", 'PRIN 07-012': "PRINCE GEORGE'S PRECINCT 07-012", 'PRIN 09-005': "PRINCE GEORGE'S PRECINCT 09-005", 'PRIN 03-001': "PRINCE GEORGE'S PRECINCT 03-001", 'PRIN 05-011': "PRINCE GEORGE'S PRECINCT 05-011", 'PRIN 07-015': "PRINCE GEORGE'S PRECINCT 07-015", 'PRIN 15-003': "PRINCE GEORGE'S PRECINCT 15-003", 'PRIN 06-011': "PRINCE GEORGE'S PRECINCT 06-011", 'PRIN 20-005': "PRINCE GEORGE'S PRECINCT 20-005", 'PRIN 21-005': "PRINCE GEORGE'S PRECINCT 21-005", 'PRIN 09-009': "PRINCE GEORGE'S PRECINCT 09-009", 'PRIN 10-010': "PRINCE GEORGE'S PRECINCT 10-010", 'PRIN 19-002': "PRINCE GEORGE'S PRECINCT 19-002", 'PRIN 12-013': "PRINCE GEORGE'S PRECINCT 12-013", 'PRIN 17-011': "PRINCE GEORGE'S PRECINCT 17-011", 'PRIN 07-023': "PRINCE GEORGE'S PRECINCT 07-023", 'PRIN 01-006': "PRINCE GEORGE'S PRECINCT 01-006", 'PRIN 12-004': "PRINCE GEORGE'S PRECINCT 12-004", 'PRIN 06-017': "PRINCE GEORGE'S PRECINCT 06-017", 'PRIN 20-009': "PRINCE GEORGE'S PRECINCT 20-009", 'PRIN 18-006': "PRINCE GEORGE'S PRECINCT 18-006", 'PRIN 17-007': "PRINCE GEORGE'S PRECINCT 17-007", 'PRIN 21-015': "PRINCE GEORGE'S PRECINCT 21-015", 'PRIN 12-006': "PRINCE GEORGE'S PRECINCT 12-006", 'PRIN 20-011': "PRINCE GEORGE'S PRECINCT 20-011", 'PRIN 10-011': "PRINCE GEORGE'S PRECINCT 10-011", 'PRIN 18-001': "PRINCE GEORGE'S PRECINCT 18-001", 'PRIN 14-011': "PRINCE GEORGE'S PRECINCT 14-011", 'PRIN 06-022': "PRINCE GEORGE'S PRECINCT 06-022", 'PRIN 21-008': "PRINCE GEORGE'S PRECINCT 21-008", 'PRIN 21-016': "PRINCE GEORGE'S PRECINCT 21-016", 'PRIN 17-002': "PRINCE GEORGE'S PRECINCT 17-002", 'PRIN 02-009': "PRINCE GEORGE'S PRECINCT 02-009", 'PRIN 06-018': "PRINCE GEORGE'S PRECINCT 06-018", 'PRIN 17-012': "PRINCE GEORGE'S PRECINCT 17-012", 'PRIN 20-003': "PRINCE GEORGE'S PRECINCT 20-003", 'PRIN 06-015': "PRINCE GEORGE'S PRECINCT 06-015", 'PRIN 12-007': "PRINCE GEORGE'S PRECINCT 12-007", 'PRIN 13-016': "PRINCE GEORGE'S PRECINCT 13-016", 'PRIN 21-009': "PRINCE GEORGE'S PRECINCT 21-009", 'PRIN 12-012': "PRINCE GEORGE'S PRECINCT 12-012", 'PRIN 17-004': "PRINCE GEORGE'S PRECINCT 17-004", 'PRIN 07-017': "PRINCE GEORGE'S PRECINCT 07-017", 'PRIN 03-005': "PRINCE GEORGE'S PRECINCT 03-005", 'PRIN 18-007': "PRINCE GEORGE'S PRECINCT 18-007", 'PRIN 13-013': "PRINCE GEORGE'S PRECINCT 13-013", 'PRIN 20-006': "PRINCE GEORGE'S PRECINCT 20-006", 'PRIN 12-001': "PRINCE GEORGE'S PRECINCT 12-001", 'PRIN 20-004': "PRINCE GEORGE'S PRECINCT 20-004", 'PRIN 06-010': "PRINCE GEORGE'S PRECINCT 06-010", 'PRIN 18-009': "PRINCE GEORGE'S PRECINCT 18-009", 'PRIN 06-004': "PRINCE GEORGE'S PRECINCT 06-004", 'PRIN 12-003': "PRINCE GEORGE'S PRECINCT 12-003", 'PRIN 12-002': "PRINCE GEORGE'S PRECINCT 12-002", 'PRIN 05-010': "PRINCE GEORGE'S PRECINCT 05-010", 'PRIN 14-013': "PRINCE GEORGE'S PRECINCT 14-013", 'PRIN 06-019': "PRINCE GEORGE'S PRECINCT 06-019", 'PRIN 13-003': "PRINCE GEORGE'S PRECINCT 13-003", 'PRIN 06-006': "PRINCE GEORGE'S PRECINCT 06-006", 'PRIN 06-005': "PRINCE GEORGE'S PRECINCT 06-005", 'PRIN 06-009': "PRINCE GEORGE'S PRECINCT 06-009", 'PRIN 14-008': "PRINCE GEORGE'S PRECINCT 14-008", 'PRIN 02-002': "PRINCE GEORGE'S PRECINCT 02-002", 'PRIN 02-010': "PRINCE GEORGE'S PRECINCT 02-010", 'PRIN 20-021': "PRINCE GEORGE'S PRECINCT 20-021", 'PRIN 21-012': "PRINCE GEORGE'S PRECINCT 21-012", 'PRIN 06-016': "PRINCE GEORGE'S PRECINCT 06-016", 'PRIN 16-002': "PRINCE GEORGE'S PRECINCT 16-002", 'PRIN 17-003': "PRINCE GEORGE'S PRECINCT 17-003", 'PRIN 20-019': "PRINCE GEORGE'S PRECINCT 20-019", 'PRIN 06-008': "PRINCE GEORGE'S PRECINCT 06-008", 'PRIN 12-008': "PRINCE GEORGE'S PRECINCT 12-008", 'PRIN 10-001': "PRINCE GEORGE'S PRECINCT 10-001", 'PRIN 09-002': "PRINCE GEORGE'S PRECINCT 09-002", 'PRIN 17-009': "PRINCE GEORGE'S PRECINCT 17-009", 'PRIN 13-004': "PRINCE GEORGE'S PRECINCT 13-004", 'PRIN 02-005': "PRINCE GEORGE'S PRECINCT 02-005", 'PRIN 19-004': "PRINCE GEORGE'S PRECINCT 19-004", 'PRIN 06-002': "PRINCE GEORGE'S PRECINCT 06-002", 'PRIN 06-023': "PRINCE GEORGE'S PRECINCT 06-023", 'PRIN 13-021': "PRINCE GEORGE'S PRECINCT 13-021", 'PRIN 21-017': "PRINCE GEORGE'S PRECINCT 21-017", 'PRIN 12-009': "PRINCE GEORGE'S PRECINCT 12-009", 'PRIN 17-005': "PRINCE GEORGE'S PRECINCT 17-005", 'PRIN 13-017': "PRINCE GEORGE'S PRECINCT 13-017", 'PRIN 17-014': "PRINCE GEORGE'S PRECINCT 17-014", 'PRIN 07-001': "PRINCE GEORGE'S PRECINCT 07-001", 'PRIN 06-027': "PRINCE GEORGE'S PRECINCT 06-027", 'PRIN 12-016': "PRINCE GEORGE'S PRECINCT 12-016", 'PRIN 06-012': "PRINCE GEORGE'S PRECINCT 06-012", 'PRIN 13-002': "PRINCE GEORGE'S PRECINCT 13-002", 'PRIN 18-011': "PRINCE GEORGE'S PRECINCT 18-011", 'PRIN 20-008': "PRINCE GEORGE'S PRECINCT 20-008", 'PRIN 18-002': "PRINCE GEORGE'S PRECINCT 18-002", 'PRIN 02-001': "PRINCE GEORGE'S PRECINCT 02-001", 'PRIN 20-007': "PRINCE GEORGE'S PRECINCT 20-007", 'PRIN 17-006': "PRINCE GEORGE'S PRECINCT 17-006", 'PRIN 06-026': "PRINCE GEORGE'S PRECINCT 06-026", 'PRIN 20-002': "PRINCE GEORGE'S PRECINCT 20-002", 'PRIN 13-024': "PRINCE GEORGE'S PRECINCT 13-024", 'PRIN 18-008': "PRINCE GEORGE'S PRECINCT 18-008", 'PRIN 06-028': "PRINCE GEORGE'S PRECINCT 06-028", 'PRIN 18-004': "PRINCE GEORGE'S PRECINCT 18-004", 'PRIN 01-008': "PRINCE GEORGE'S PRECINCT 01-008", 'PRIN 20-012': "PRINCE GEORGE'S PRECINCT 20-012", 'PRIN 16-001': "PRINCE GEORGE'S PRECINCT 16-001", 'PRIN 07-018': "PRINCE GEORGE'S PRECINCT 07-018", 'PRIN 07-021': "PRINCE GEORGE'S PRECINCT 07-021", 'PRIN 18-005': "PRINCE GEORGE'S PRECINCT 18-005", 'PRIN 07-024': "PRINCE GEORGE'S PRECINCT 07-024", 'PRIN 14-015': "PRINCE GEORGE'S PRECINCT 14-015", 'PRIN 13-023': "PRINCE GEORGE'S PRECINCT 13-023", 'PRIN 17-008': "PRINCE GEORGE'S PRECINCT 17-008", 'PRIN 18-003': "PRINCE GEORGE'S PRECINCT 18-003", 'PRIN 20-010': "PRINCE GEORGE'S PRECINCT 20-010", 'PRIN 13-008': "PRINCE GEORGE'S PRECINCT 13-008", 'PRIN 09-003': "PRINCE GEORGE'S PRECINCT 09-003", 'PRIN 06-024': "PRINCE GEORGE'S PRECINCT 06-024", 'PRIN 13-001': "PRINCE GEORGE'S PRECINCT 13-001", 'PRIN 18-013': "PRINCE GEORGE'S PRECINCT 18-013", 'PRIN 13-020': "PRINCE GEORGE'S PRECINCT 13-020", 'PRIN 06-021': "PRINCE GEORGE'S PRECINCT 06-021", 'PRIN 06-029': "PRINCE GEORGE'S PRECINCT 06-029", 'PRIN 05-009': "PRINCE GEORGE'S PRECINCT 05-009", 'PRIN 07-020': "PRINCE GEORGE'S PRECINCT 07-020", 'PRIN 11-005': "PRINCE GEORGE'S PRECINCT 11-005", 'PRIN 13-018': "PRINCE GEORGE'S PRECINCT 13-018", 'PRIN 18-010': "PRINCE GEORGE'S PRECINCT 18-010", 'PRIN 12-017': "PRINCE GEORGE'S PRECINCT 12-017", 'PRIN 20-013': "PRINCE GEORGE'S PRECINCT 20-013", 'PRIN 06-014': "PRINCE GEORGE'S PRECINCT 06-014", 'PRIN 20-018': "PRINCE GEORGE'S PRECINCT 20-018", 'PRIN 20-020': "PRINCE GEORGE'S PRECINCT 20-020", 'PRIN 03-008': "PRINCE GEORGE'S PRECINCT 03-008", 'PRIN 13-022': "PRINCE GEORGE'S PRECINCT 13-022", 'PRIN 20-016': "PRINCE GEORGE'S PRECINCT 20-016", 'PRIN 10-014': "PRINCE GEORGE'S PRECINCT 10-014", 'PRIN 12-018': "PRINCE GEORGE'S PRECINCT 12-018", 'PRIN 14-016': "PRINCE GEORGE'S PRECINCT 14-016", 'PRIN 03-007': "PRINCE GEORGE'S PRECINCT 03-007", 'PRIN 06-025': "PRINCE GEORGE'S PRECINCT 06-025", 'PRIN 20-014': "PRINCE GEORGE'S PRECINCT 20-014", 'PRIN 13-025': "PRINCE GEORGE'S PRECINCT 13-025", 'PRIN 17-015': "PRINCE GEORGE'S PRECINCT 17-015", 'PRIN 09-012': "PRINCE GEORGE'S PRECINCT 09-012", 'PRIN 17-013': "PRINCE GEORGE'S PRECINCT 17-013", 'PRIN 18-014': "PRINCE GEORGE'S PRECINCT 18-014", 'PRIN 07-019': "PRINCE GEORGE'S PRECINCT 07-019", 'PRIN 16-005': "PRINCE GEORGE'S PRECINCT 16-005", 'PRIN 18-012': "PRINCE GEORGE'S PRECINCT 18-012", 'PRIN 20-015': "PRINCE GEORGE'S PRECINCT 20-015", 'PRIN 21-019': "PRINCE GEORGE'S PRECINCT 21-019", 'PRIN 13-019': "PRINCE GEORGE'S PRECINCT 13-019", 'PRIN 18-015': "PRINCE GEORGE'S PRECINCT 18-015", 'PRIN 14-014': "PRINCE GEORGE'S PRECINCT 14-014", 'PRIN 19-005': "PRINCE GEORGE'S PRECINCT 19-005", 'PRIN 18-016': "PRINCE GEORGE'S PRECINCT 18-016", 'PRIN 14-017': "PRINCE GEORGE'S PRECINCT 14-017", 'PRIN 02-096': "PRINCE GEORGE'S PRECINCT 02-096", 'PRIN 02-097': "PRINCE GEORGE'S PRECINCT 02-097", 'PRIN 02-098': "PRINCE GEORGE'S PRECINCT 02-098", 'PRIN 02-099': "PRINCE GEORGE'S PRECINCT 02-099", 'PRIN 06-097': "PRINCE GEORGE'S PRECINCT 06-097", 'PRIN 06-098': "PRINCE GEORGE'S PRECINCT 06-098", 'PRIN 06-099': "PRINCE GEORGE'S PRECINCT 06-099", 'PRIN 07-099': "PRINCE GEORGE'S PRECINCT 07-099", 'PRIN 09-014': "PRINCE GEORGE'S PRECINCT 09-014", 'PRIN 09-015': "PRINCE GEORGE'S PRECINCT 09-015", 'PRIN 09-099': "PRINCE GEORGE'S PRECINCT 09-099", 'PRIN 13-099': "PRINCE GEORGE'S PRECINCT 13-099", 'PRIN 14-099': "PRINCE GEORGE'S PRECINCT 14-099", 'PRIN 16-098': "PRINCE GEORGE'S PRECINCT 16-098", 'PRIN 16-099': "PRINCE GEORGE'S PRECINCT 16-099", 'PRIN 18-099': "PRINCE GEORGE'S PRECINCT 18-099", 'PRIN 19-098': "PRINCE GEORGE'S PRECINCT 19-098", 'PRIN 19-099': "PRINCE GEORGE'S PRECINCT 19-099", 'PRIN 20-017': "PRINCE GEORGE'S PRECINCT 20-017", 'PRIN 20-097': "PRINCE GEORGE'S PRECINCT 20-097", 'PRIN 20-098': "PRINCE GEORGE'S PRECINCT 20-098", 'PRIN 20-099': "PRINCE GEORGE'S PRECINCT 20-099", 'PRIN 21-091': "PRINCE GEORGE'S PRECINCT 21-091", 'PRIN 21-092': "PRINCE GEORGE'S PRECINCT 21-092", 'PRIN 21-093': "PRINCE GEORGE'S PRECINCT 21-093", 'PRIN 21-094': "PRINCE GEORGE'S PRECINCT 21-094", 'PRIN 21-095': "PRINCE GEORGE'S PRECINCT 21-095", 'PRIN 21-096': "PRINCE GEORGE'S PRECINCT 21-096", 'PRIN 21-097': "PRINCE GEORGE'S PRECINCT 21-097", 'PRIN 21-098': "PRINCE GEORGE'S PRECINCT 21-098", 'PRIN 21-099': "PRINCE GEORGE'S PRECINCT 21-099", 'QUEE 03-002': "QUEEN ANNE'S PRECINCT 03-002", 'QUEE 05-002': "QUEEN ANNE'S PRECINCT 05-002", 'QUEE 04-002': "QUEEN ANNE'S PRECINCT 04-002", 'QUEE 04-003': "QUEEN ANNE'S PRECINCT 04-003", 'QUEE 02-001': "QUEEN ANNE'S PRECINCT 02-001", 'QUEE 04-006': "QUEEN ANNE'S PRECINCT 04-006", 'QUEE 04-001': "QUEEN ANNE'S PRECINCT 04-001", 'QUEE 01-001': "QUEEN ANNE'S PRECINCT 01-001", 'QUEE 05-001': "QUEEN ANNE'S PRECINCT 05-001", 'QUEE 07-001': "QUEEN ANNE'S PRECINCT 07-001", 'QUEE 06-001': "QUEEN ANNE'S PRECINCT 06-001", 'SOME 08-001': 'SOMERSET PRECINCT 08-001', 'SOME 07-001': 'SOMERSET PRECINCT 07-001', 'SOME 03-001': 'SOMERSET PRECINCT 03-001', 'SOME 01-002': 'SOMERSET PRECINCT 01-002', 'SOME 05-001': 'SOMERSET PRECINCT 05-001', 'SOME 04-001': 'SOMERSET PRECINCT 04-001', 'SOME 12-001': 'SOMERSET PRECINCT 12-001', 'SOME 08-002': 'SOMERSET PRECINCT 08-002', 'SOME 14-001': 'SOMERSET PRECINCT 14-001', 'SOME 01-001': 'SOMERSET PRECINCT 01-001', 'SOME 15-001': 'SOMERSET PRECINCT 15-001', 'SOME 09-001': 'SOMERSET PRECINCT 09-001', 'SOME 06-001': 'SOMERSET PRECINCT 06-001', 'SOME 02-001': 'SOMERSET PRECINCT 02-001', 'SOME 13-002': 'SOMERSET PRECINCT 13-002', 'SOME 15-002': 'SOMERSET PRECINCT 15-002', 'SOME 10-001': 'SOMERSET PRECINCT 10-001', 'SOME 13-003': 'SOMERSET PRECINCT 13-003', 'SOME 13-001': 'SOMERSET PRECINCT 13-001', 'SOME 11-001': 'SOMERSET PRECINCT 11-001', 'SOME 01-003': 'SOMERSET PRECINCT 01-003', 'SOME 15-003': 'SOMERSET PRECINCT 15-003', 'SOME 10-002': 'SOMERSET PRECINCT 10-002', 'STMA 05-001': "ST MARY'S PRECINCT 05-001", 'STMA 06-003': "ST MARY'S PRECINCT 06-003", 'STMA 04-001': "ST MARY'S PRECINCT 04-001", 'STMA 05-002': "ST MARY'S PRECINCT 05-002", 'STMA 05-003': "ST MARY'S PRECINCT 05-003", 'STMA 07-001': "ST MARY'S PRECINCT 07-001", 'STMA 04-003': "ST MARY'S PRECINCT 04-003", 'STMA 01-001': "ST MARY'S PRECINCT 01-001", 'STMA 02-001': "ST MARY'S PRECINCT 02-001", 'STMA 08-004': "ST MARY'S PRECINCT 08-004", 'STMA 06-002': "ST MARY'S PRECINCT 06-002", 'STMA 03-002': "ST MARY'S PRECINCT 03-002", 'STMA 03-003': "ST MARY'S PRECINCT 03-003", 'STMA 08-005': "ST MARY'S PRECINCT 08-005", 'STMA 01-002': "ST MARY'S PRECINCT 01-002", 'STMA 03-001': "ST MARY'S PRECINCT 03-001", 'STMA 08-003': "ST MARY'S PRECINCT 08-003", 'STMA 03-004': "ST MARY'S PRECINCT 03-004", 'STMA 04-002': "ST MARY'S PRECINCT 04-002", 'STMA 03-005': "ST MARY'S PRECINCT 03-005", 'STMA 02-002': "ST MARY'S PRECINCT 02-002", 'STMA 08-001': "ST MARY'S PRECINCT 08-001", 'STMA 08-007': "ST MARY'S PRECINCT 08-007", 'STMA 08-002': "ST MARY'S PRECINCT 08-002", 'STMA 08-006': "ST MARY'S PRECINCT 08-006", 'STMA 06-004': "ST MARY'S PRECINCT 06-004", 'STMA 08-009': "ST MARY'S PRECINCT 08-009", 'STMA 08-011': "ST MARY'S PRECINCT 08-011", 'STMA 06-001': "ST MARY'S PRECINCT 06-001", 'STMA 03-006': "ST MARY'S PRECINCT 03-006", 'STMA 02-003': "ST MARY'S PRECINCT 02-003", 'STMA 02-004': "ST MARY'S PRECINCT 02-004", 'STMA 08-010': "ST MARY'S PRECINCT 08-010", 'STMA 08-008': "ST MARY'S PRECINCT 08-008", 'STMA 08-012': "ST MARY'S PRECINCT 08-012", 'STMA 09-001': "ST MARY'S PRECINCT 09-001", 'TALB 04-001': 'TALBOT PRECINCT 04-001', 'TALB 01-002': 'TALBOT PRECINCT 01-002', 'TALB 03-001': 'TALBOT PRECINCT 03-001', 'TALB 01-001': 'TALBOT PRECINCT 01-001', 'TALB 01-005': 'TALBOT PRECINCT 01-005', 'TALB 02-001': 'TALBOT PRECINCT 02-001', 'TALB 01-003': 'TALBOT PRECINCT 01-003', 'TALB 02-003': 'TALBOT PRECINCT 02-003', 'TALB 03-003': 'TALBOT PRECINCT 03-003', 'TALB 05-002': 'TALBOT PRECINCT 05-002', 'TALB 05-001': 'TALBOT PRECINCT 05-001', 'TALB 02-002': 'TALBOT PRECINCT 02-002', 'WASH 23-000': 'WASHINGTON PRECINCT 23-001', 'WASH 02-000': 'WASHINGTON PRECINCT 02-001', 'WASH 04-000': 'WASHINGTON PRECINCT 04-001', 'WASH 01-000': 'WASHINGTON PRECINCT 01-001', 'WASH 13-001': 'WASHINGTON PRECINCT 13-001', 'WASH 16-000': 'WASHINGTON PRECINCT 16-001', 'WASH 07-001': 'WASHINGTON PRECINCT 07-001', 'WASH 27-002': 'WASHINGTON PRECINCT 27-002', 'WASH 06-002': 'WASHINGTON PRECINCT 06-002', 'WASH 05-000': 'WASHINGTON PRECINCT 05-001', 'WASH 27-001': 'WASHINGTON PRECINCT 27-001', 'WASH 10-002': 'WASHINGTON PRECINCT 10-002', 'WASH 06-001': 'WASHINGTON PRECINCT 06-001', 'WASH 18-001': 'WASHINGTON PRECINCT 18-001', 'WASH 19-000': 'WASHINGTON PRECINCT 19-001', 'WASH 12-000': 'WASHINGTON PRECINCT 12-001', 'WASH 09-000': 'WASHINGTON PRECINCT 09-001', 'WASH 07-002': 'WASHINGTON PRECINCT 07-002', 'WASH 10-001': 'WASHINGTON PRECINCT 10-001', 'WASH 08-000': 'WASHINGTON PRECINCT 08-001', 'WASH 26-004': 'WASHINGTON PRECINCT 26-004', 'WASH 20-000': 'WASHINGTON PRECINCT 20-001', 'WASH 18-002': 'WASHINGTON PRECINCT 18-002', 'WASH 26-001': 'WASHINGTON PRECINCT 26-001', 'WASH 17-001': 'WASHINGTON PRECINCT 17-001', 'WASH 21-001': 'WASHINGTON PRECINCT 21-001', 'WASH 26-002': 'WASHINGTON PRECINCT 26-002', 'WASH 15-000': 'WASHINGTON PRECINCT 15-001', 'WASH 18-003': 'WASHINGTON PRECINCT 18-003', 'WASH 21-002': 'WASHINGTON PRECINCT 21-002', 'WASH 13-002': 'WASHINGTON PRECINCT 13-002', 'WASH 25-004': 'WASHINGTON PRECINCT 25-004', 'WASH 25-002': 'WASHINGTON PRECINCT 25-002', 'WASH 10-003': 'WASHINGTON PRECINCT 10-003', 'WASH 26-003': 'WASHINGTON PRECINCT 26-003', 'WASH 22-002': 'WASHINGTON PRECINCT 22-002', 'WASH 11-001': 'WASHINGTON PRECINCT 11-001', 'WASH 21-003': 'WASHINGTON PRECINCT 21-003', 'WASH 03-003': 'WASHINGTON PRECINCT 03-003', 'WASH 14-001': 'WASHINGTON PRECINCT 14-001', 'WASH 14-002': 'WASHINGTON PRECINCT 14-002', 'WASH 10-004': 'WASHINGTON PRECINCT 10-004', 'WASH 03-002': 'WASHINGTON PRECINCT 03-002', 'WASH 22-001': 'WASHINGTON PRECINCT 22-001', 'WASH 11-002': 'WASHINGTON PRECINCT 11-002', 'WASH 24-001': 'WASHINGTON PRECINCT 24-001', 'WASH 03-004': 'WASHINGTON PRECINCT 03-004', 'WASH 03-001': 'WASHINGTON PRECINCT 03-001', 'WASH 25-001': 'WASHINGTON PRECINCT 25-001', 'WASH 18-004': 'WASHINGTON PRECINCT 18-004', 'WASH 25-003': 'WASHINGTON PRECINCT 25-003', 'WASH 10-005': 'WASHINGTON PRECINCT 10-005', 'WASH 03-005': 'WASHINGTON PRECINCT 03-005', 'WICO 09-006': 'WICOMICO COUNTY PRECINCT 09-006', 'WICO 05-005': 'WICOMICO COUNTY PRECINCT 05-005', 'WICO 08-002': 'WICOMICO COUNTY PRECINCT 08-002', 'WICO 13-003': 'WICOMICO COUNTY PRECINCT 13-003', 'WICO 02-001': 'WICOMICO COUNTY PRECINCT 02-001', 'WICO 11-004': 'WICOMICO COUNTY PRECINCT 11-004', 'WICO 04-003': 'WICOMICO COUNTY PRECINCT 04-003', 'WICO 05-004': 'WICOMICO COUNTY PRECINCT 05-004', 'WICO 07-001': 'WICOMICO COUNTY PRECINCT 07-001', 'WICO 15-001': 'WICOMICO COUNTY PRECINCT 15-001', 'WICO 14-002': 'WICOMICO COUNTY PRECINCT 14-002', 'WICO 17-001': 'WICOMICO COUNTY PRECINCT 17-001', 'WICO 05-006': 'WICOMICO COUNTY PRECINCT 05-006', 'WICO 16-003': 'WICOMICO COUNTY PRECINCT 16-003', 'WICO 05-007': 'WICOMICO COUNTY PRECINCT 05-007', 'WICO 05-008': 'WICOMICO COUNTY PRECINCT 05-008', 'WICO 08-004': 'WICOMICO COUNTY PRECINCT 08-004', 'WICO 01-003': 'WICOMICO COUNTY PRECINCT 01-003', 'WICO 16-005': 'WICOMICO COUNTY PRECINCT 16-005', 'WICO 15-003': 'WICOMICO COUNTY PRECINCT 15-003', 'WICO 05-003': 'WICOMICO COUNTY PRECINCT 05-003', 'WICO 08-001': 'WICOMICO COUNTY PRECINCT 08-001', 'WICO 13-002': 'WICOMICO COUNTY PRECINCT 13-002', 'WICO 10-003': 'WICOMICO COUNTY PRECINCT 10-003', 'WICO 06-001': 'WICOMICO COUNTY PRECINCT 06-001', 'WICO 09-008': 'WICOMICO COUNTY PRECINCT 09-008', 'WICO 01-002': 'WICOMICO COUNTY PRECINCT 01-002', 'WICO 09-007': 'WICOMICO COUNTY PRECINCT 09-007', 'WICO 08-005': 'WICOMICO COUNTY PRECINCT 08-005', 'WICO 04-002': 'WICOMICO COUNTY PRECINCT 04-002', 'WICO 05-001': 'WICOMICO COUNTY PRECINCT 05-001', 'WICO 16-004': 'WICOMICO COUNTY PRECINCT 16-004', 'WICO 13-004': 'WICOMICO COUNTY PRECINCT 13-004', 'WICO 09-001': 'WICOMICO COUNTY PRECINCT 09-001', 'WICO 16-001': 'WICOMICO COUNTY PRECINCT 16-001', 'WICO 05-009': 'WICOMICO COUNTY PRECINCT 05-009', 'WICO 09-005': 'WICOMICO COUNTY PRECINCT 09-005', 'WICO 14-001': 'WICOMICO COUNTY PRECINCT 14-001', 'WICO 04-001': 'WICOMICO COUNTY PRECINCT 04-001', 'WICO 08-003': 'WICOMICO COUNTY PRECINCT 08-003', 'WICO 16-006': 'WICOMICO COUNTY PRECINCT 16-006', 'WICO 16-002': 'WICOMICO COUNTY PRECINCT 16-002', 'WICO 15-002': 'WICOMICO COUNTY PRECINCT 15-002', 'WICO 09-003': 'WICOMICO COUNTY PRECINCT 09-003', 'WICO 13-001': 'WICOMICO COUNTY PRECINCT 13-001', 'WICO 05-002': 'WICOMICO COUNTY PRECINCT 05-002', 'WICO 11-002': 'WICOMICO COUNTY PRECINCT 11-002', 'WICO 11-003': 'WICOMICO COUNTY PRECINCT 11-003', 'WICO 09-002': 'WICOMICO COUNTY PRECINCT 09-002', 'WICO 10-001': 'WICOMICO COUNTY PRECINCT 10-001', 'WICO 01-001': 'WICOMICO COUNTY PRECINCT 01-001', 'WICO 10-002': 'WICOMICO COUNTY PRECINCT 10-002', 'WICO 09-004': 'WICOMICO COUNTY PRECINCT 09-004', 'WICO 11-001': 'WICOMICO COUNTY PRECINCT 11-001', 'WORC 07-001': 'WORCESTER PRECINCT 07-001', 'WORC 01-001': 'WORCESTER PRECINCT 01-001', 'WORC 03-001': 'WORCESTER PRECINCT 03-001', 'WORC 06-001': 'WORCESTER PRECINCT 06-001', 'WORC 05-002': 'WORCESTER PRECINCT 05-002', 'WORC 04-001': 'WORCESTER PRECINCT 04-001', 'WORC 05-001': 'WORCESTER PRECINCT 05-001', 'WORC 06-003': 'WORCESTER PRECINCT 06-003', 'WORC 03-002': 'WORCESTER PRECINCT 03-002', 'WORC 04-004': 'WORCESTER PRECINCT 04-004', 'WORC 06-002': 'WORCESTER PRECINCT 06-002', 'WORC 01-002': 'WORCESTER PRECINCT 01-002', 'WORC 02-005': 'WORCESTER PRECINCT 02-005', 'WORC 04-002': 'WORCESTER PRECINCT 04-002', 'WORC 03-003': 'WORCESTER PRECINCT 03-003', 'WORC 02-002': 'WORCESTER PRECINCT 02-002', 'WORC 02-003': 'WORCESTER PRECINCT 02-003', 'WORC 02-001': 'WORCESTER PRECINCT 02-001', 'WORC 04-003': 'WORCESTER PRECINCT 04-003', 'WORC 02-004': 'WORCESTER PRECINCT 02-004'}

#Apply the changes
allocated_precinct["join_col"] =allocated_precinct["join_col"].map(source_vest_id_changes_dict).fillna(allocated_precinct["join_col"])

#Create the join col in VEST
vest_md_18["join_col"]=vest_md_18["NAME"]

In [ ]:
#Join the two datasets
final_join = pd.merge(vest_md_18,allocated_precinct,how="outer",on="join_col",indicator=True)
final_join["_merge"].value_counts()

In [ ]:
#Confrim the 11 unjoined precincts all have 0 votes
empty_precincts = []
for index, row in final_join[final_join["_merge"]=="left_only"].iterrows():
    number_of_zero_vote_races = 0
    for race in data_columns:
        if (row[race+"_x"]==0):
            number_of_zero_vote_races += 1
    if(number_of_zero_vote_races == len(data_columns)):
        empty_precincts.append(row["join_col"])
print(len(empty_precincts))

## Run the precinct-by-precinct check

In [ ]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [ ]:
precinct_votes_check(final_join[final_join["_merge"]=="both"],data_columns,True,"join_col",print_level=0)

From above:

>There are  2034  total rows  
715  of these rows have election result differences  
1319  of these rows are the same  

>The max difference between any one shared column in a row is:  2.0  
The average difference is:  1.0107296137339057  
There are  0 precinct results with a difference greater than 10  

Given how small they are, these differences all likely stem from allocation


## Shapefiles

It doesn't look like it's the exact same shapefile, but fairly close, some precincts may have been split

In [ ]:
md_pct = gp.read_file("/Users/peterhorton/Downloads/precinct2010/Precinct2010_83NadMeters.shp")

In [ ]:
md_pct = md_pct.to_crs(vest_md_18.crs)


ax = md_pct.boundary.plot(figsize=(20,10))
vest_md_18.boundary.plot(color='red',ax=ax)